In [ ]:
out = True

In [ ]:
special_treat = ['HQHZ8','HQYH7','HQHT6','HQYH6','HQHF7','HQHF4','HQYH1','HQYH2','HQLX1','HQGX3','HQYH11','HQHT11','HQHZ13','HQYH16',\
                 'HQHZ12','HQCT4','HQHT13','HQPA2','HQYH25','HQYH23','HQZX3','HQCC3','HQZT1','HQGJ1','HQHT16','HQGS2','HQYH27',\
                 'HQHT20','HQHT19',]
new_treat = []
new_treat_model = []

In [ ]:
use_excel = 1

In [ ]:
#import
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import numpy as np
import pymysql
import operator
import os
from scipy import stats
from statsmodels.regression.linear_model import OLS
from datetime import *
import scipy.io as sio
import h5py
import copy
import shutil
import rarfile
from general_function import *
import scipy.stats as scs
from matplotlib_venn import venn3
from matplotlib_venn import venn2
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['SimHei']  #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  #用来正常显示负号
sns.set(style="whitegrid")
myfont = FontProperties(fname=r'C:\Windows\Fonts\simhei.ttf', size=10)
sns.set(
    font=myfont.get_name(),
    style="whitegrid",
    palette="muted",
    color_codes=True)

In [ ]:
#today
today = datetime.today()
year = str(today.year)
month = today.month
if month < 10:
    month = '0' + str(month)
else:
    month = str(month)
day = today.day
if day < 10:
    day = '0' + str(day)
else:
    day = str(day)    
today_nodot = year + month + day
today = year + '.' + month + '.' + day
today_for_mat = today[-5:]
int_today = int(today.replace('.',''))
int_yesterday = int(get_n_week_date_before(today_nodot))

In [ ]:
today,today_for_mat,int_today,today_nodot,int_yesterday

today = '2019.08.07'
today_for_mat = '08.07'
int_today = 20190807
today_nodot = '20190807'
int_yesterday = 20190806

In [ ]:
if not use_excel:
    print('in track')
    os.chdir(r'C:/Users/bj/Documents/WeChat Files/C10623912/FileStorage/File/2019-'+today.split('.')[1]+'/')
    for mat_rar_file in ['A~D mat ','G mat ','E，F mat文件 ','H mat ','I mat ','J mat ','k mat ']:
        mat_rar_file = mat_rar_file + '('+today_for_mat+').rar'
        f = rarfile.RarFile(mat_rar_file)
        f.extractall('C:/Users/bj/Documents/WeChat Files/C10623912/FileStorage/File/2019-'+today.split('.')[1]+'/')
    os.chdir('原始mat文件/')
    for file in [file for file in os.listdir()]:
        #print(file)
        shutil.copy2(file, r'F:/金湖回测系统/6日循环/要测文件/')
    os.chdir(r'C:/Users/bj/Documents/WeChat Files/C10623912/FileStorage/File/2019-'+today.split('.')[1]+'/')
    del_file('原始mat文件')
    os.rmdir('原始mat文件')

In [ ]:
#FULL CONT 读取mat文件里的选股记录
os.chdir(r'F:\金湖回测系统\6日循环\要测文件')
filenames = listdir(file_type='.mat')
filenames_ = [[i.split('.\\')[-1].split('_')[0],i[5:9],0] if '万' not in i else \
              [i.split('.\\')[-1].split('_')[0],i[5:9],i.split('万')[0][-4:]] for i in filenames]
def process(df):
    return df.index.tolist()
by_model = pd.DataFrame(filenames_).groupby([0,2]).apply(process)
full_cont = {}
version_num = {}
for (model,limit) in by_model.index:
    if model in ['A2','A3','B4','C3','E2','F1']:
        model_list = by_model.loc[(model,limit)]
        filenames_model = []
        for i in model_list:
            filenames_model.append(filenames[i])
        already_date = []
        already_rank = []
        for name in sorted(filenames_model)[::-1]:
            os.chdir(r'F:\金湖回测系统\6日循环\要测文件')
            try:
                file = sio.loadmat(name)
                date_in_this_file = [i[0][0] for i in file['table'][1]]
                if name == sorted(filenames_model)[0] or date_in_this_file[0] >= 737392:
                    new = [i for i in date_in_this_file if i not in already_date][::-1]
                    new_index = [index for index,value in enumerate(date_in_this_file) if value not in already_date][::-1]
                else:
                    new = [i for i in date_in_this_file if i not in already_date][::-1][:-1]
                    new_index = [index for index,value in enumerate(date_in_this_file) if value not in already_date][::-1][:-1]
                old_length = len(already_date)
                already_date += new
                new_length = len(already_date)
                for i in new_index:
                    already_rank.append([j[0] for j in file['table'][0][i]])
            except:
                pass
        cont = pd.concat([pd.Series(already_date),pd.Series(already_rank)],axis=1).sort_values(0).reset_index(drop=True)
        cont = cont.set_index(0)[1]
        cont.index = [int(str(date.fromordinal(int(cci)-366)).replace('-','')) for cci in cont.index]
        version_num[model] = sorted(filenames_model)[-1].split('_')[1]
        full_cont[model] = cont
    elif model in ['I1']:
        model_list = by_model.loc[(model,limit)]
        filenames_model = []
        for i in model_list:
            filenames_model.append(filenames[i])
        already_date = []
        already_rank = []
        for name in sorted(filenames_model):
            os.chdir(r'F:\金湖回测系统\6日循环\要测文件')
            file = sio.loadmat(name)
            date_in_this_file = [i[0][0] for i in file['table'][1]]
            new = [i for i in date_in_this_file if i not in already_date][::-1]
            new_index = [index for index,value in enumerate(date_in_this_file) if value not in already_date][::-1]
            old_length = len(already_date)
            already_date += new
            new_length = len(already_date)
            for i in new_index:
                already_rank.append([j[0] for j in file['table'][0][i]])
        cont = pd.concat([pd.Series(already_date),pd.Series(already_rank)],axis=1).sort_values(0).reset_index(drop=True)
        cont = cont.set_index(0)[1]
        cont.index = [int(str(date.fromordinal(int(cci)-366)).replace('-','')) for cci in cont.index]
        version_num[model] = sorted(filenames_model)[-1].split('_')[1]
        full_cont[model] = cont
        
        model_list = by_model.loc[(model,limit)]
        filenames_model = []
        for i in model_list:
            filenames_model.append(filenames[i])
        already_date = []
        already_rank = []
        for name in sorted(filenames_model)[::-1]:
            os.chdir(r'F:\金湖回测系统\6日循环\要测文件')
            try:
                file = sio.loadmat(name)
                date_in_this_file = [i[0][0] for i in file['table'][1]]
                new = [i for i in date_in_this_file if i not in already_date][::-1]
                new_index = [index for index,value in enumerate(date_in_this_file) if value not in already_date][::-1]
                old_length = len(already_date)
                already_date += new
                new_length = len(already_date)
                for i in new_index:
                    already_rank.append([j[0] for j in file['table'][0][i]])
            except:
                pass
        cont = pd.concat([pd.Series(already_date),pd.Series(already_rank)],axis=1).sort_values(0).reset_index(drop=True)
        cont = cont.set_index(0)[1]
        cont.index = [int(str(date.fromordinal(int(cci)-366)).replace('-','')) for cci in cont.index]
        version_num[model+'new'] = sorted(filenames_model)[-1].split('_')[1]
        full_cont[model+'new'] = cont
    else:
        model_list = by_model.loc[(model,limit)]
        filenames_model = []
        for i in model_list:
            filenames_model.append(filenames[i])
        already_date = []
        already_rank = []
        for name in sorted(filenames_model)[::-1]:
            os.chdir(r'F:\金湖回测系统\6日循环\要测文件')
            try:
                file = sio.loadmat(name)
                date_in_this_file = [i[0][0] for i in file['table'][1]]
                new = [i for i in date_in_this_file if i not in already_date][::-1]
                new_index = [index for index,value in enumerate(date_in_this_file) if value not in already_date][::-1]
                old_length = len(already_date)
                already_date += new
                new_length = len(already_date)
                for i in new_index:
                    already_rank.append([j[0] for j in file['table'][0][i]])
            except:
                pass
        cont = pd.concat([pd.Series(already_date),pd.Series(already_rank)],axis=1).sort_values(0).reset_index(drop=True)
        cont = cont.set_index(0)[1]
        cont.index = [int(str(date.fromordinal(int(cci)-366)).replace('-','')) for cci in cont.index]
        version_num[model] = sorted(filenames_model)[-1].split('_')[1]
        full_cont[model] = cont

In [ ]:
#J2 有一个文件有重复日期，给删了
full_cont['J2'] = full_cont['J2'].reset_index().drop_duplicates(subset = ['index']).set_index('index')[1]

In [ ]:
if use_excel:
    print('in track')
    os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')
    full_cont['A3'].loc[int_today] = pd.read_csv('prospect_A3.csv',header = None).iloc[:,-1].tolist()
    full_cont['C2'].loc[int_today] = pd.read_csv('prospect_C2.csv',header = None).iloc[:,-1].tolist()
    full_cont['C4'].loc[int_today] = pd.read_csv('prospect_C4.csv',header = None).iloc[:,-1].tolist()
    full_cont['B3'].loc[int_today] = pd.read_csv('prospect_B3.csv',header = None).iloc[:,-1].tolist()
    full_cont['B4'].loc[int_today] = pd.read_csv('prospect_B4.csv',header = None).iloc[:,-1].tolist()
    full_cont['I4'].loc[int_today] = pd.read_csv('prospect_I4.csv',header = None).iloc[:,-1].tolist()
    full_cont['I5'].loc[int_today] = pd.read_csv('prospect_I5.csv',header = None).iloc[:,-1].tolist()
    full_cont['J2'].loc[int_today] = pd.read_csv('prospect_J2.csv',header = None).iloc[:,-1].tolist()

In [ ]:
#data
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')
industry_full = pd.read_excel('sw_industry_full.xlsx',index_col = 0)
industry = industry_full.loc[[int_yesterday]] if int_yesterday in industry_full.index.tolist() else industry_full.loc[[20190211]]
industry_ind = industry.T
industry_ind.columns = ['ind']
STcode_by_date = pd.read_csv('STcode_by_date_for_generator.csv',index_col=0)
mkt_cap_ard = pd.read_csv('mkt_cap_ard.csv',index_col=0)
mkt_cap_ard.columns = [int(i) for i in mkt_cap_ard.columns]
mkt_cap_ard.fillna(method='ffill',inplace=True)
amt = pd.read_csv('amt.csv',index_col=0)
amt.columns = [int(i) for i in amt.columns]
close = pd.read_csv('close.csv',index_col=0)
close.columns = [int(i) for i in close.columns]
code_to_name = pd.read_csv('code_to_name.csv',index_col=0,encoding='gbk')['SEC_NAME']
code_to_name.index = [int(i[:6]) for i in code_to_name.index]
hs300members = pd.read_csv('hs300members.csv',index_col=0).T
hs300members.index = [int(i) for i in hs300members.index]
adjfactor = pd.read_csv('adjfactor.csv',index_col=0)
adjfactor.columns = [int(i) for i in adjfactor.columns]
free_float_shares = pd.read_csv('free_float_shares.csv',index_col=0)
free_float_shares.columns = [int(i) for i in free_float_shares.columns]
adjusted_weight_coef = free_float_shares * close
total_shares = pd.read_csv('total_shares.csv',index_col=0)
total_shares.columns = [int(i) for i in total_shares.columns]
prest_kick = pd.read_csv('prest_kick.csv',index_col = 0)
prest_kick.columns = [int(i) for i in prest_kick.columns]

In [ ]:
#rongzirongquan
os.chdir(r'C:\Users\bj\Desktop\刘泽丹\Code')
huafu = pd.read_excel('华福证券'+today_nodot+'.xlsx')
yinhe = pd.read_excel('银河证券'+today_nodot+'.xlsx')
haitong = pd.read_excel('海通证券'+today_nodot+'.xlsx')
lianxun = pd.read_excel('联讯证券'+today_nodot+'.xlsx')
guoxin = pd.read_excel('国信证券'+today_nodot+'.xlsx')
guoxin['折算率'] = 0.6
huatai = pd.read_excel('华泰证券'+today_nodot+'.xlsx')
pingan = pd.read_excel('平安证券'+today_nodot+'.xlsx')
guosheng = pd.read_excel('国盛证券'+today_nodot+'.xlsx')
caitong = pd.read_excel('财通证券'+today_nodot+'.xlsx')
caitong['折算率'] = 0.6
zhongxin = pd.read_excel('中信证券'+today_nodot+'.xlsx')
changcheng = pd.read_excel('长城证券'+today_nodot+'.xlsx')
data = pd.read_csv('tsdata_'+str(int_yesterday)+'.csv',encoding='gbk')
all_stocks = [int(i[-6:]) for i in data['代码']]
subyinhe = yinhe.set_index('证券代码')['折算率']
subyinhe.name = '银河'
subyinhe = subyinhe[subyinhe.index.isin(all_stocks)].index.tolist()
subhuafu = huafu.set_index('证券代码')['折算率']
subhuafu.name = '华福'
subhuafu = subhuafu[subhuafu.index.isin(all_stocks)].index.tolist()
subhaitong = haitong.set_index('证券代码')['本期折算率']
subhaitong.name = '海通'
subhaitong = subhaitong[subhaitong.index.isin(all_stocks)].index.tolist() + [755]
sublianxun = lianxun.set_index('股票代码')['折算率']
sublianxun.name = '联讯'
sublianxun = sublianxun[sublianxun.index.isin(all_stocks)].index.tolist()
subguoxin = guoxin.set_index('证券代码')['折算率']
subguoxin.name = '国信'
subguoxin = subguoxin[subguoxin.index.isin(all_stocks)].index.tolist()
subhuatai = huatai.set_index('证券代码')['担保折算率']
subhuatai.name = '华泰'
subhuatai = subhuatai[subhuatai.index.isin(all_stocks)].index.tolist()
subpingan = pingan.set_index('证券代码')['担保折算率']
subpingan.name = '平安'
subpingan = subpingan[subpingan.index.isin(all_stocks)].index.tolist()
subcaitong = caitong.set_index('证券代码')['折算率']
subcaitong.name = '财通'
subcaitong = subcaitong[subcaitong.index.isin(all_stocks)].index.tolist()
subchangcheng = changcheng.set_index('股票代码')['折算率']
subchangcheng.name = '长城'
subchangcheng = subchangcheng[subchangcheng.index.isin(all_stocks)].index.tolist()
subzhongxin = zhongxin.set_index('证券代码')['担保折算率']
subzhongxin.name = '中信'
subzhongxin = [szx for szx in subzhongxin[subzhongxin.index.isin(all_stocks)].index.tolist() if szx not in [600016]]
subguosheng = guosheng.set_index('证券代码')['担保折算率']
subguosheng.name = '国盛'
subguosheng = subguosheng[subguosheng.index.isin(all_stocks)].index.tolist()

In [ ]:
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')
full_cont['I4'].loc[20190822] = pd.read_csv('prospect_I4_20190822.csv',header = None).iloc[:,-1].tolist()
full_cont['I5'].loc[20190822] = pd.read_csv('prospect_I5_20190822.csv',header = None).iloc[:,-1].tolist()

full_cont['I4'].loc[20190821] = pd.read_csv('prospect_I4_20190821.csv',header = None).iloc[:,-1].tolist()
full_cont['I5'].loc[20190821] = pd.read_csv('prospect_I5_20190821.csv',header = None).iloc[:,-1].tolist()

full_cont['I4'].loc[20190820] = pd.read_csv('prospect_I4_20190820.csv',header = None).iloc[:,-1].tolist()
full_cont['I4'].loc[20190819] = pd.read_csv('prospect_I4_20190819.csv',header = None).iloc[:,-1].tolist()
full_cont['I4'].loc[20190816] = pd.read_csv('prospect_I4_20190816.csv',header = None).iloc[:,-1].tolist()
full_cont['I4'].loc[20190815] = pd.read_csv('prospect_I4_20190815.csv',header = None).iloc[:,-1].tolist()
full_cont['I4'].loc[20190814] = pd.read_csv('prospect_I4_20190814.csv',header = None).iloc[:,-1].tolist()
full_cont['I4'].loc[20190813] = pd.read_csv('prospect_I4_20190813.csv',header = None).iloc[:,-1].tolist()
full_cont['I4'].loc[20190812] = pd.read_csv('prospect_I4_20190812.csv',header = None).iloc[:,-1].tolist()
full_cont['I5'].loc[20190820] = pd.read_csv('prospect_I5_20190820.csv',header = None).iloc[:,-1].tolist()
full_cont['I5'].loc[20190819] = pd.read_csv('prospect_I5_20190819.csv',header = None).iloc[:,-1].tolist()
full_cont['I5'].loc[20190816] = pd.read_csv('prospect_I5_20190816.csv',header = None).iloc[:,-1].tolist()
full_cont['I5'].loc[20190815] = pd.read_csv('prospect_I5_20190815.csv',header = None).iloc[:,-1].tolist()
full_cont['I5'].loc[20190814] = pd.read_csv('prospect_I5_20190814.csv',header = None).iloc[:,-1].tolist()
full_cont['I5'].loc[20190813] = pd.read_csv('prospect_I5_20190813.csv',header = None).iloc[:,-1].tolist()
full_cont['I5'].loc[20190812] = pd.read_csv('prospect_I5_20190812.csv',header = None).iloc[:,-1].tolist()

In [ ]:
#正常买的模型

In [ ]:
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')
try:
    os.chdir('自用篮子组合'+str(int_today))
except FileNotFoundError:
    print('new directory created!')
    os.mkdir('自用篮子组合'+str(int_today))

In [ ]:
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')
account_info = pd.read_excel('account_info.xlsx',index_col = 2)
account_info = account_info.loc[:,['账户备注',int_today]].dropna()
capital = pd.read_excel('账户资金'+today_nodot+'.xlsx',index_col = 0)
checking_account_info = account_info.copy().reset_index()
checking_account_info['产品标识'] = pd.Series(info[0] for info in checking_account_info['产品标识'].str.split(','))
checking_account_info.set_index('产品标识', inplace = True)

In [ ]:
needed_code = []
for i in capital.index.unique():
    print(capital.loc[[i]]['名称'].iloc[0])
    print('+'.join(capital.loc[[i]]['买入股票'].tolist()).replace(' ','_').replace('__','_').replace('+',' + '))
    try:
        print(' + '.join(sorted(checking_account_info[int_today].xs(i).replace(' * 20%', '').replace(' * 80%', '').replace(' * 66.6%', '')\
              .replace(' * 33.3%', '').replace(' * 30%', '').replace(' * 70%', '').replace(' * 50%', '')\
              .replace(' * 100%', '').replace('(','').replace(')','').split(' + '))))
    except:
        print('###############################################')
        print(capital.loc[[i]]['名称'].tolist(),i)
    print()
    needed_code.append(i)
checking_account_info = checking_account_info.loc[needed_code]

In [ ]:
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')

In [ ]:
changing_model_portfolio = np.array(pd.read_excel('补仓账户资金'+today_nodot+'.xlsx')).tolist()

In [ ]:
changing_model_portfolio

In [ ]:
#正常买的部分

In [ ]:
full_txt_df = pd.DataFrame()
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/自用篮子组合'+str(int_today)+'/')

In [ ]:
for account_code in checking_account_info.index:
    if int_today in prest_kick.index.tolist():
        today_prest_kick = prest_kick.xs(int_today).dropna().tolist()
    else:
        today_prest_kick = []
    print(today_prest_kick)
    account_name = checking_account_info.loc[account_code,'账户备注']
    account_str = checking_account_info.loc[account_code,int_today].split('+')
    if account_code in special_treat:
        special_flag = 1
        if account_name in ['金湖宏观量化一期','金湖一号','金湖AI对冲十三期','德聚仁和三号','红墙泰和对冲六号','对冲二十五期']:
            keeping = subhaitong
            dealer = '海通'
        elif account_name in ['于微','金湖AI对冲十八期']:
            keeping = subhuafu
            dealer = '华福'
        elif account_name in ['金湖AI对冲五十期','金湖AI对冲二十二期','九期龙哥','金湖增强四期','金湖AI对冲十期','无量稳健六号',\
                              '私享五号信用','金湖中性对冲六号','红墙泰和指数增强七期']:
            keeping = subyinhe
            dealer = '银河'
        elif account_name in ['']:
            keeping = subguoxin
            dealer = '国信'
        elif account_name in ['金湖AI对冲五十七期']:
            keeping = subguosheng
            dealer = '国盛'
        elif account_name in ['圭源金湖启航','无量稳健二号','无量稳健二号分仓']:
            keeping = subhuatai
            dealer = '华泰'
        elif account_name in ['金湖聚乐盈1号','金湖聚乐盈1号(旧)']:
            keeping = subchangcheng
            dealer = '长城'
        elif account_name in ['圭源腾飞10号信用']:
            keeping = subpingan
            dealer = '平安'
        elif account_name in ['金湖AI对冲四十五期','金湖AI对冲四十五期分仓']:
            keeping = sublianxun
            dealer = '联讯'
        elif account_name in ['私享八号']:
            keeping = subcaitong
            dealer = '财通'
        elif account_name in ['金湖AI对冲十五期']:
            keeping = subzhongxin
            dealer = '中信'
        else:
            print('#######################################################################################')
    else:
        special_flag = 0
        keeping = all_stocks
        dealer = '非信用'
    STcode = [int(stock) for stock in STcode_by_date.xs(int_yesterday).dropna().tolist()]
    print(account_name)
    for single_account_str in account_str:
        model = single_account_str.split('(')[-1].split('_')[0]
        if preset_hold_days(model) == 2:
            continue
        universe = preset_hold_days(model, return_universe = 'long')[1]
        if '市值' in single_account_str:
            divider = 2; multiplier = 0
            special = '市值'
            first_n = int(single_account_str.split('支')[0].split('_')[-2])
            ind_num = int(single_account_str.split('支')[0].split('_')[-3])
            ranked_stock_universe = pd.Series(full_cont[model].xs(int_today))[\
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(STcode))].tolist()
            yester_mkt_cap = mkt_cap_ard.xs(int_yesterday).loc[ranked_stock_universe]
            def aa(df):
                def standardize(ts):
                    ts = ts.copy().dropna()
                    return (ts - ts.mean()) / ts.std()
                return(standardize(df.iloc[:,1]).sort_values(ascending=False))
            ranked_with_industry_ind = industry_ind.loc[ranked_stock_universe]
            standardized_yester_mkt_cap = pd.concat([ranked_with_industry_ind,yester_mkt_cap],axis=1).groupby('ind').apply(aa)
            standardized_yester_mkt_cap.fillna(0,inplace=True)
            ranked_with_industry_ind['rank'] = range(len(ranked_with_industry_ind))
            portfolio = []
            #for ind in ranked_with_industry_ind.groupby('ind').mean().sort_values('rank').index.tolist()[:first_n]:
            for ind in ((ranked_with_industry_ind.drop_duplicates(subset = ['ind'], keep = 'first').set_index('ind')*multiplier + \
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').mean())['rank']).loc[\
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').count()['rank'][\
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').count()['rank']>=ind_num
                ].index.tolist()].sort_values().index.tolist()[:first_n]:
                if account_code in ['HQHX1',] and int_today >= 20190718:
                    print('HQHX1')
                    portfolio += ranked_with_industry_ind[(ranked_with_industry_ind.index.isin(keeping))&\
                        (ranked_with_industry_ind['ind'] == ind)&\
                        (~ranked_with_industry_ind.index.isin(today_prest_kick))&\
                        (~ranked_with_industry_ind.index.isin([18,601258,600255,600518,587,\
                        603101,600172,600221,600515,600555,600568,600856,506,603619,\
                        600525,600119,835,981,600069,600715,662,600112,760,600651,600086,2606,408,\
                        600083,600179,806,752,600891,600462,820,600687,600615,600747,982,600485,2477,571,\
                        600634,971,600701,600240,939,48,600614,600666,600807,600289]))].iloc[:ind_num].index.tolist()
                else:
                    portfolio += ranked_with_industry_ind[(ranked_with_industry_ind.index.isin(keeping))&\
                        (ranked_with_industry_ind['ind'] == ind)&\
                        (~ranked_with_industry_ind.index.isin(today_prest_kick))].iloc[:ind_num].index.tolist()
            portfolio_origin = []
            #for ind in ranked_with_industry_ind.groupby('ind').mean().sort_values('rank').index.tolist()[:first_n]:
            for ind in ((ranked_with_industry_ind.drop_duplicates(subset = ['ind'], keep = 'first').set_index('ind')*multiplier + \
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').mean())['rank']).loc[\
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').count()['rank'][\
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').count()['rank']>=ind_num
                ].index.tolist()].sort_values().index.tolist()[:first_n]:
                portfolio_origin += ranked_with_industry_ind[(ranked_with_industry_ind['ind'] == ind)].iloc[:ind_num].index.tolist()
            standardized_yester_mkt_cap = standardized_yester_mkt_cap.reset_index().set_index('level_1').loc[portfolio]
            standardized_yester_mkt_cap['quantile'] = scs.norm.cdf(standardized_yester_mkt_cap.reset_index().set_index('level_1').\
                loc[portfolio].iloc[:,1])
            portfolio_weight = pd.Series([stock_weight/standardized_yester_mkt_cap['quantile'].sum() \
                for stock_weight in standardized_yester_mkt_cap['quantile']],index=portfolio)
            while(True):
                if portfolio_weight.max() / portfolio_weight.min() <= 3:
                    break
                else:
                    portfolio_weight = portfolio_weight - (portfolio_weight - portfolio_weight.mean())/2
            df = industry_ind.loc[portfolio]
            df['weight'] = portfolio_weight
            df['indcode'] = industry_ind.loc[df.index]
            df['name'] = code_to_name.loc[df.index]
            df.reset_index(inplace=True)
            df = df.loc[:,['name','index','ind','indcode','weight']]
            df.columns = ['股票名称','股票代码','所属行业','行业代码','资金占比']
            txt_df = pd.DataFrame()
            txt_df['证券代码'] = ['0'*(6-len(str(i)))+str(i) for i in df['股票代码']]
            txt_df['证券名称'] = df['股票名称']
            txt_df['每股份数'] = 100.00
            txt_df['金额比例'] = [round(i*100,4) for i in df['资金占比']]
            txt_df[''] = ''
            print('共计：',df['股票代码'].isin(keeping).sum(),'   受影响：',([stock for stock in portfolio_origin if stock not in portfolio]),\
                '   替换为：',([stock for stock in portfolio if stock not in portfolio_origin]))
            if special_flag == 1:
                generated_str = model+'_'+version_num[model]+'_'+universe+'_'+today_nodot+'_市值行业_'+str(ind_num)+'_'+str(first_n)+'（'+account_name+'）'+'.txt'
                print(dealer)
            else:
                generated_str = model+'_'+version_num[model]+'_'+universe+'_'+today_nodot+'_市值行业_'+str(ind_num)+'_'+str(first_n)+'.txt'
            print(generated_str)
        elif '原味' in single_account_str:
            special = '原味'
            buy_num = int(single_account_str.split('支')[0].split('_')[-1])
            amt_limitation = 0
            if amt_limitation != 0:
                yester_amt = amt.xs(int_yesterday).fillna(0)
                yester_amt = yester_amt[~(yester_amt>amt_limitation)].index.tolist()
            else:
                yester_amt = []
            if account_code in ['HQHX1',] and int_today >= 20190718:
                print('HQHX1')
                ranked_stock_universe = pd.Series(full_cont[model].xs(int_today))[\
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(STcode)) & \
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(yester_amt)) & \
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(today_prest_kick)) & \
                                        (pd.Series(full_cont[model].xs(int_today)).isin(keeping)) & \
                                        (~pd.Series(full_cont[model].xs(int_today)).isin([18,601258,600255,600518,587,\
                                                    603101,600172,600221,600515,600555,600568,600856,506,603619,\
                                                    600525,600119,835,981,600069,600715,662,600112,760,600651,600086,2606,408,\
                                                    600083,600179,806,752,600891,600462,820,600687,600615,600747,982,600485,2477,571,\
                                                    600634,971,600701,600240,939,48,600614,600666,600807,600289]))].tolist()
            else:
                ranked_stock_universe = pd.Series(full_cont[model].xs(int_today))[\
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(STcode)) & \
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(yester_amt)) & \
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(today_prest_kick)) & \
                                        (pd.Series(full_cont[model].xs(int_today)).isin(keeping))].tolist()
            portfolio = ranked_stock_universe[:buy_num]
            ranked_stock_universe_origin = pd.Series(full_cont[model].xs(int_today))[\
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(STcode)) & \
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(yester_amt))].tolist()
            portfolio_origin = ranked_stock_universe_origin[:buy_num]
            portfolio_weight = pd.Series([1/len(portfolio)]*len(portfolio),index=portfolio)
            df = industry_ind.loc[portfolio]
            df['weight'] = portfolio_weight
            df['indcode'] = industry_ind.loc[df.index]
            df['name'] = code_to_name.loc[df.index]
            df.reset_index(inplace=True)
            df = df.loc[:,['name','index','ind','indcode','weight']]
            df.columns = ['股票名称','股票代码','所属行业','行业代码','资金占比']
            txt_df = pd.DataFrame()
            txt_df['证券代码'] = ['0'*(6-len(str(i)))+str(i) for i in df['股票代码']]
            txt_df['证券名称'] = df['股票名称']
            txt_df['每股份数'] = 100.00
            txt_df['金额比例'] = [round(i*100,4) for i in df['资金占比']]
            txt_df[''] = ''
            print('共计：',df['股票代码'].isin(keeping).sum(),'   受影响：',([stock for stock in portfolio_origin if stock not in portfolio]),\
                      '   替换为：',([stock for stock in portfolio if stock not in portfolio_origin]))
            if special_flag == 1:
                if amt_limitation == 0:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支（原汁原味）'+'_'+today_nodot+'（'+account_name+'）'+'.txt'
                else:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支'+'_'+today_nodot+'（'+account_name+'）'+'.txt'
                print(dealer)
            else:
                if amt_limitation == 0:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支（原汁原味）'+'_'+today_nodot+'.txt'
                else:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支'+'_'+today_nodot+'.txt'
            print(generated_str)
        elif '沪深' in single_account_str:
            special = '沪深'
            buy_num = int(single_account_str.split('支')[0].split('_')[-1])
            amt_limitation = 50000000
            if amt_limitation != 0:
                yester_amt = amt.xs(int_yesterday).fillna(0)
                yester_amt = yester_amt[~(yester_amt>amt_limitation)].index.tolist()
            else:
                yester_amt = []
            ranked_stock_universe =  pd.Series(full_cont[model].xs(int_today))[\
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(STcode)) & \
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(yester_amt))].tolist()
            origin = ranked_stock_universe[:buy_num]
            #HS300行业权重
            members = hs300members.xs(int_yesterday).tolist()
            mid_members_industry = (pd.concat([(\
                adjusted_weight_coef.xs(int_yesterday).loc[members]/\
                adjusted_weight_coef.xs(int_yesterday).loc[members]\
                .sum()).sort_values(),industry_ind['ind'].loc[members]],axis=1).groupby('ind').sum().iloc[:,0]*100)\
                .sort_values(ascending=False)
            #ranked_stock_universe个股行业
            ranked_with_industry = industry_ind.loc[ranked_stock_universe].dropna()
            ranked_with_industry['rrank'] = range(len(ranked_with_industry))
            exp = ranked_with_industry.sort_values('rrank').dropna()
            exp['rank'] = range(1,len(exp)+1)
            adj_list = []
            name_list = []
            for included_ind in exp['ind'].unique():
                sub = exp[exp['ind'] == included_ind]
                adjust = (exp['rank'].mean() - sub['rank'].mean()) * len(sub)
                adj_list.append(adjust)
                name_list.append(included_ind)
            adj_list = [i/np.max(adj_list) * 20 for i in adj_list]
            members_industry = pd.DataFrame(list(zip(name_list,adj_list))).set_index(0)[1] + mid_members_industry
            members_industry = members_industry[members_industry>0]
            members_industry = members_industry/members_industry.sum()
            members_industry = round(members_industry.sort_values(ascending=False)*buy_num)
            res = pd.concat([industry_ind.loc[origin]['ind'].value_counts()/buy_num * 100,\
                       members_industry/members_industry.sum() * 100,\
                       mid_members_industry],axis=1)
            res.columns = ['origin','adjusted','hs300']
            res.fillna(0,inplace=True)
            res['result'] = (res['origin'] - res['adjusted']) * 1/2 + res['adjusted']
            members_industry = round(res['result']/res['result'].sum() * buy_num)
            portfolio = []
            for indname in members_industry.index:
                add = ranked_with_industry[(ranked_with_industry['ind'] == indname) &\
                    (ranked_with_industry.index.isin(keeping)) & (~ranked_with_industry.index.isin(today_prest_kick))].\
                    index.tolist()[:int(members_industry.xs(indname))]
                portfolio += add
            portfolio_origin = []
            for indname in members_industry.index:
                add = ranked_with_industry[(ranked_with_industry['ind'] == indname)].\
                    index.tolist()[:int(members_industry.xs(indname))]
                portfolio_origin += add
            p_data = adjusted_weight_coef.xs(int_yesterday).loc[portfolio]
            portfolio_weight = p_data/(p_data.sum())
            portfolio_weight = (portfolio_weight - (portfolio_weight - portfolio_weight.mean())/2).sort_values()
            df = industry_ind.loc[portfolio]
            df['weight'] = portfolio_weight
            df['indcode'] = industry_ind.loc[df.index]
            df['name'] = code_to_name.loc[df.index]
            df.reset_index(inplace=True)
            df = df.loc[:,['name','index','ind','indcode','weight']]
            df.columns = ['股票名称','股票代码','所属行业','行业代码','资金占比']
            txt_df = pd.DataFrame()
            txt_df['证券代码'] = ['0'*(6-len(str(i)))+str(i) for i in df['股票代码']]
            txt_df['证券名称'] = df['股票名称']
            txt_df['每股份数'] = 100.00
            txt_df['金额比例'] = [round(i*100,4) for i in df['资金占比']]
            txt_df[''] = ''
            print('共计：',df['股票代码'].isin(keeping).sum(),'   受影响：',([stock for stock in portfolio_origin if stock not in portfolio]),\
                      '   替换为：',([stock for stock in portfolio if stock not in portfolio_origin]))
            if special_flag == 1:
                generated_str = model+'_'+version_num[model]+'_'+universe+'_'+today_nodot+'_HS300增强_'+str(buy_num)+'（'+account_name+'）'+'.txt'
                print(dealer)
            else:
                generated_str = model+'_'+version_num[model]+'_'+universe+'_'+today_nodot+'_HS300增强_'+str(buy_num)+'.txt'
            print(generated_str)
        else:
            special = ''
            buy_num = int(single_account_str.split('支')[0].split('_')[-1])
            amt_limitation = 50000000
            if out:
                ranked_stock_universe = pd.Series(full_cont[model].xs(int_today))[
                                            (~pd.Series(full_cont[model].xs(int_today)).isin(STcode)) & \
                                            (pd.Series(full_cont[model].xs(int_today)).isin(keeping))].tolist()
            else:
                if amt_limitation != 0:
                    yester_amt = amt.xs(int_yesterday).fillna(0)
                    yester_amt = yester_amt[~(yester_amt>amt_limitation)].index.tolist()
                else:
                    yester_amt = []
                if account_code in ['HQHX1',] and int_today >= 20190718:
                    print('HQHX1')
                    ranked_stock_universe = pd.Series(full_cont[model].xs(int_today))[\
                                            (~pd.Series(full_cont[model].xs(int_today)).isin(STcode)) & \
                                            (~pd.Series(full_cont[model].xs(int_today)).isin(yester_amt)) & \
                                            (~pd.Series(full_cont[model].xs(int_today)).isin(today_prest_kick)) & \
                                            (pd.Series(full_cont[model].xs(int_today)).isin(keeping)) & \
                                            (~pd.Series(full_cont[model].xs(int_today)).isin([18,601258,600255,600518,587,\
                                                        603101,600172,600221,600515,600555,600568,600856,506,603619,\
                                                        600525,600119,835,981,600069,600715,662,600112,760,600651,600086,2606,408,\
                                                        600083,600179,806,752,600891,600462,820,600687,600615,600747,982,600485,2477,571,\
                                                        600634,971,600701,600240,939,48,600614,600666,600807,600289]))].tolist()
                else:
                    ranked_stock_universe = pd.Series(full_cont[model].xs(int_today))[\
                                            (~pd.Series(full_cont[model].xs(int_today)).isin(STcode)) & \
                                            (~pd.Series(full_cont[model].xs(int_today)).isin(yester_amt)) & \
                                            (~pd.Series(full_cont[model].xs(int_today)).isin(today_prest_kick)) & \
                                            (pd.Series(full_cont[model].xs(int_today)).isin(keeping))].tolist()
            portfolio = ranked_stock_universe[:buy_num]
            portfolio_weight = pd.Series([1/len(portfolio)]*len(portfolio),index=portfolio)
            df = industry_ind.loc[portfolio]
            df['weight'] = portfolio_weight
            df['indcode'] = industry_ind.loc[df.index]
            df['name'] = code_to_name.loc[df.index]
            df.reset_index(inplace=True)
            df = df.loc[:,['name','index','ind','indcode','weight']]
            df.columns = ['股票名称','股票代码','所属行业','行业代码','资金占比']
            txt_df = pd.DataFrame()
            txt_df['证券代码'] = ['0'*(6-len(str(i)))+str(i) for i in df['股票代码']]
            txt_df['证券名称'] = df['股票名称']
            txt_df['每股份数'] = 100.00
            txt_df['金额比例'] = [round(i*100,4) for i in df['资金占比']]
            txt_df[''] = ''
            if special_flag == 1:
                if amt_limitation == 0:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支（原汁原味）'+'_'+today_nodot+'（'+account_name+'）'+'.txt'
                else:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支'+'_'+today_nodot+'（'+account_name+'）'+'.txt'
                print(dealer)
            else:
                if amt_limitation == 0:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支（原汁原味）'+'_'+today_nodot+'.txt'
                else:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支'+'_'+today_nodot+'.txt'
            print(generated_str)
        model = single_account_str.split('(')[-1].split('_')[0]
        txt_df.to_csv(generated_str, encoding = 'gbk', index=False)
        txt_df['产品标识'] = account_code
        model_capital = capital.loc[[account_code]]
        if special == '':
            model_capital = model_capital[(~((model_capital['买入股票'].str.contains('沪深')) | \
                                            (model_capital['买入股票'].str.contains('市值')) | \
                                            (model_capital['买入股票'].str.contains('原味')))) & \
                                            (model_capital['买入股票'].str[:2] == model)]['要买资金'][account_code]
        else:
            model_capital = model_capital[(model_capital['买入股票'].str.contains(special)) & (model_capital['买入股票'].str[:2] == model)]['要买资金'][account_code]
        txt_df['任务资金'] = round(txt_df['金额比例']/100 * model_capital)
        txt_df['模型'] = model
        full_txt_df = pd.concat([full_txt_df, txt_df])
        print(type(model_capital), model_capital)
        print()

In [ ]:
for changing_model_portfolio_info in changing_model_portfolio:
    account_name = changing_model_portfolio_info[1]
    account_code = changing_model_portfolio_info[0]
    account_str = changing_model_portfolio_info[2].split('+')
    changing_model_date = changing_model_portfolio_info[3]
    changing_model_date_yesterday = int(get_n_week_date_before(str(changing_model_date)))
    if changing_model_date == 20190308 and account_code != 'HQYH9':
        changing_model_date_yesterday = 20190311
    changing_model_capital = changing_model_portfolio_info[4]
    changing_model_marker = changing_model_portfolio_info[5]
    if account_code in special_treat:
        special_flag = 1
        if account_name in ['金湖宏观量化一期','金湖一号','金湖AI对冲十三期','德聚仁和三号','红墙泰和对冲六号','对冲二十五期']:
            keeping = subhaitong
            dealer = '海通'
        elif account_name in ['于微','金湖AI对冲十八期']:
            keeping = subhuafu
            dealer = '华福'
        elif account_name in ['金湖AI对冲五十期','金湖AI对冲二十二期','九期龙哥','金湖增强四期','金湖AI对冲十期','无量稳健六号',\
                              '私享五号信用','金湖中性对冲六号','红墙泰和指数增强七期']:
            keeping = subyinhe
            dealer = '银河'
        elif account_name in ['']:
            keeping = subguoxin
            dealer = '国信'
        elif account_name in ['金湖AI对冲五十七期']:
            keeping = subguosheng
            dealer = '国盛'
        elif account_name in ['圭源金湖启航','金湖聚乐盈1号','无量稳健二号','无量稳健二号分仓']:
            keeping = subhuatai
            dealer = '华泰'
        elif account_name in ['圭源腾飞10号信用']:
            keeping = subpingan
            dealer = '平安'
        elif account_name in ['金湖AI对冲四十五期','金湖AI对冲四十五期分仓']:
            keeping = sublianxun
            dealer = '联讯'
        elif account_name in ['私享八号']:
            keeping = subcaitong
            dealer = '财通'
        elif account_name in ['金湖AI对冲十五期']:
            keeping = subzhongxin
            dealer = '中信'
        else:
            print('#######################################################################################')
    else:
        special_flag = 0
        keeping = all_stocks
        dealer = '非信用'
    if int_today in prest_kick.index.tolist():
        today_prest_kick = prest_kick.xs(int_today).dropna().tolist()
    else:
        today_prest_kick = []
    print(today_prest_kick)
    ###
    STcode = [int(stock) for stock in STcode_by_date.xs(int_yesterday).dropna().tolist()]
    print(account_name)
    for single_account_str in account_str:
        print(today_prest_kick)
        model = single_account_str.split('(')[-1].split('_')[0]
        universe = preset_hold_days(model, return_universe = 'long')[1]
        if '市值' in single_account_str:
            divider = 2; multiplier = 0
            special = '市值'
            first_n = int(single_account_str.split('支')[0].split('_')[-2])
            ind_num = int(single_account_str.split('支')[0].split('_')[-3])
            ranked_stock_universe = pd.Series(full_cont[model].xs(changing_model_date))[\
                                        (~pd.Series(full_cont[model].xs(changing_model_date)).isin(STcode))].tolist()
            yester_mkt_cap = mkt_cap_ard.xs(changing_model_date_yesterday).loc[ranked_stock_universe]
            def aa(df):
                def standardize(ts):
                    ts = ts.copy().dropna()
                    return (ts - ts.mean()) / ts.std()
                return(standardize(df.iloc[:,1]).sort_values(ascending=False))
            ranked_with_industry_ind = industry_ind.loc[ranked_stock_universe]
            standardized_yester_mkt_cap = pd.concat([ranked_with_industry_ind,yester_mkt_cap],axis=1).groupby('ind').apply(aa)
            standardized_yester_mkt_cap.fillna(0,inplace=True)
            ranked_with_industry_ind['rank'] = range(len(ranked_with_industry_ind))
            portfolio = []
            #for ind in ranked_with_industry_ind.groupby('ind').mean().sort_values('rank').index.tolist()[:first_n]:
            for ind in ((ranked_with_industry_ind.drop_duplicates(subset = ['ind'], keep = 'first').set_index('ind')*multiplier + \
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').mean())['rank']).loc[\
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').count()['rank'][\
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').count()['rank']>=ind_num
                ].index.tolist()].sort_values().index.tolist()[:first_n]:
                if account_code in ['HQHX1',] and int_today >= 20190718:
                    print('HQHX1')
                    portfolio += ranked_with_industry_ind[(ranked_with_industry_ind.index.isin(keeping))&\
                        (ranked_with_industry_ind['ind'] == ind)&
                        (~ranked_with_industry_ind.index.isin(today_prest_kick))&
                        (~ranked_with_industry_ind.index.isin([18,601258,600255,600518,587,\
                        603101,600172,600221,600515,600555,600568,600856,506,603619,\
                        600525,600119,835,981,600069,600715,662,600112,760,600651,600086,2606,408,\
                        600083,600179,806,752,600891,600462,820,600687,600615,600747,982,600485,2477,571,\
                        600634,971,600701,600240,939,48,600614,600666,600807,600289]))].iloc[:ind_num].index.tolist()
                else:
                    portfolio += ranked_with_industry_ind[(ranked_with_industry_ind.index.isin(keeping))&\
                        (ranked_with_industry_ind['ind'] == ind)&
                        (~ranked_with_industry_ind.index.isin(today_prest_kick))].iloc[:ind_num].index.tolist()
            portfolio_origin = []
            #for ind in ranked_with_industry_ind.groupby('ind').mean().sort_values('rank').index.tolist()[:first_n]:
            for ind in ((ranked_with_industry_ind.drop_duplicates(subset = ['ind'], keep = 'first').set_index('ind')*multiplier + \
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').mean())['rank']).loc[\
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').count()['rank'][\
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').count()['rank']>=ind_num
                ].index.tolist()].sort_values().index.tolist()[:first_n]:
                portfolio_origin += ranked_with_industry_ind[(ranked_with_industry_ind['ind'] == ind)].iloc[:ind_num].index.tolist()
            standardized_yester_mkt_cap = standardized_yester_mkt_cap.reset_index().set_index('level_1').loc[portfolio]
            standardized_yester_mkt_cap['quantile'] = scs.norm.cdf(standardized_yester_mkt_cap.reset_index().set_index('level_1').\
                loc[portfolio].iloc[:,1])
            portfolio_weight = pd.Series([stock_weight/standardized_yester_mkt_cap['quantile'].sum() \
                for stock_weight in standardized_yester_mkt_cap['quantile']],index=portfolio)
            while(True):
                if portfolio_weight.max() / portfolio_weight.min() <= 3:
                    break
                else:
                    portfolio_weight = portfolio_weight - (portfolio_weight - portfolio_weight.mean())/2
            df = industry_ind.loc[portfolio]
            df['weight'] = portfolio_weight
            df['indcode'] = industry_ind.loc[df.index]
            df['name'] = code_to_name.loc[df.index]
            df.reset_index(inplace=True)
            df = df.loc[:,['name','index','ind','indcode','weight']]
            df.columns = ['股票名称','股票代码','所属行业','行业代码','资金占比']
            txt_df = pd.DataFrame()
            txt_df['证券代码'] = ['0'*(6-len(str(i)))+str(i) for i in df['股票代码']]
            txt_df['证券名称'] = df['股票名称']
            txt_df['每股份数'] = 100.00
            txt_df['金额比例'] = [round(i*100,4) for i in df['资金占比']]
            txt_df[''] = ''
            print('共计：',df['股票代码'].isin(keeping).sum(),'   受影响：',([stock for stock in portfolio_origin if stock not in portfolio]),\
                      '   替换为：',([stock for stock in portfolio if stock not in portfolio_origin]))
            if special_flag == 1:
                generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(changing_model_date)+'_市值行业_'+str(ind_num)+'_'+str(first_n)+'（'+account_name+'）'+'.txt'
                print(dealer)
            else:
                generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(changing_model_date)+'_市值行业_'+str(ind_num)+'_'+str(first_n)+'.txt'
            print(generated_str)
        elif '原味' in single_account_str:
            special = '原味'
            buy_num = int(single_account_str.split('支')[0].split('_')[-1])
            amt_limitation = 0
            if amt_limitation != 0:
                yester_amt = amt.xs(changing_model_date_yesterday).fillna(0)
                yester_amt = yester_amt[~(yester_amt>amt_limitation)].index.tolist()
            else:
                yester_amt = []
            if account_code in ['HQHX1',] and int_today >= 20190718:
                print('HQHX1')
                ranked_stock_universe = pd.Series(full_cont[model].xs(changing_model_date))[\
                                        (~pd.Series(full_cont[model].xs(changing_model_date)).isin(STcode)) & \
                                        (~pd.Series(full_cont[model].xs(changing_model_date)).isin(yester_amt)) & \
                                        (~pd.Series(full_cont[model].xs(changing_model_date)).isin(today_prest_kick)) & \
                                        (pd.Series(full_cont[model].xs(changing_model_date)).isin(keeping)) & \
                                        (~pd.Series(full_cont[model].xs(changing_model_date)).isin([18,601258,600255,600518,587,\
                                                    603101,600172,600221,600515,600555,600568,600856,506,603619,\
                                                    600525,600119,835,981,600069,600715,662,600112,760,600651,600086,2606,408,\
                                                    600083,600179,806,752,600891,600462,820,600687,600615,600747,982,600485,2477,571,\
                                                    600634,971,600701,600240,939,48,600614,600666,600807,600289]))].tolist()
            else:
                ranked_stock_universe = pd.Series(full_cont[model].xs(changing_model_date))[\
                                        (~pd.Series(full_cont[model].xs(changing_model_date)).isin(STcode)) & \
                                        (~pd.Series(full_cont[model].xs(changing_model_date)).isin(yester_amt)) & \
                                        (~pd.Series(full_cont[model].xs(changing_model_date)).isin(today_prest_kick)) & \
                                        (pd.Series(full_cont[model].xs(changing_model_date)).isin(keeping))].tolist()
            portfolio = ranked_stock_universe[:buy_num]
            ranked_stock_universe_origin = [st for st in full_cont[model].xs(changing_model_date) if st not in STcode \
                                            and st not in yester_amt]
            portfolio_origin = ranked_stock_universe_origin[:buy_num]
            portfolio_weight = pd.Series([1/len(portfolio)]*len(portfolio),index=portfolio)
            df = industry_ind.loc[portfolio]
            df['weight'] = portfolio_weight
            df['indcode'] = industry_ind.loc[df.index]
            df['name'] = code_to_name.loc[df.index]
            df.reset_index(inplace=True)
            df = df.loc[:,['name','index','ind','indcode','weight']]
            df.columns = ['股票名称','股票代码','所属行业','行业代码','资金占比']
            txt_df = pd.DataFrame()
            txt_df['证券代码'] = ['0'*(6-len(str(i)))+str(i) for i in df['股票代码']]
            txt_df['证券名称'] = df['股票名称']
            txt_df['每股份数'] = 100.00
            txt_df['金额比例'] = [round(i*100,4) for i in df['资金占比']]
            txt_df[''] = ''
            print('共计：',df['股票代码'].isin(keeping).sum(),'   受影响：',([stock for stock in portfolio_origin if stock not in portfolio]),\
                      '   替换为：',([stock for stock in portfolio if stock not in portfolio_origin]))
            if special_flag == 1:
                if amt_limitation == 0:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支（原汁原味）'+'_'+str(changing_model_date)+'（'+account_name+'）'+'.txt'
                else:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支'+'_'+str(changing_model_date)+'（'+account_name+'）'+'.txt'
                print(dealer)
            else:
                if amt_limitation == 0:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支（原汁原味）'+'_'+str(changing_model_date)+'.txt'
                else:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支'+'_'+str(changing_model_date)+'.txt'
            print(generated_str)
        elif '沪深' in single_account_str:
            special = '沪深'
            buy_num = int(single_account_str.split('支')[0].split('_')[-1])
            amt_limitation = 50000000
            if amt_limitation != 0:
                yester_amt = amt.xs(changing_model_date_yesterday).fillna(0)
                yester_amt = yester_amt[~(yester_amt>amt_limitation)].index.tolist()
            else:
                yester_amt = []
            ranked_stock_universe = pd.Series(full_cont[model].xs(changing_model_date))[\
                                    (~pd.Series(full_cont[model].xs(changing_model_date)).isin(STcode)) & \
                                    (~pd.Series(full_cont[model].xs(changing_model_date)).isin(yester_amt))].tolist()
            origin = ranked_stock_universe[:buy_num]
            #HS300行业权重
            members = hs300members.xs(changing_model_date_yesterday).tolist()
            mid_members_industry = (pd.concat([(\
                adjusted_weight_coef.xs(changing_model_date_yesterday).loc[members]/\
                adjusted_weight_coef.xs(changing_model_date_yesterday).loc[members]\
                .sum()).sort_values(),industry_ind['ind'].loc[members]],axis=1).groupby('ind').sum().iloc[:,0]*100)\
                .sort_values(ascending=False)
            #ranked_stock_universe个股行业
            ranked_with_industry = industry_ind.loc[ranked_stock_universe].dropna()
            ranked_with_industry['rrank'] = range(len(ranked_with_industry))
            exp = ranked_with_industry.sort_values('rrank').dropna()
            exp['rank'] = range(1,len(exp)+1)
            adj_list = []
            name_list = []
            for included_ind in exp['ind'].unique():
                sub = exp[exp['ind'] == included_ind]
                adjust = (exp['rank'].mean() - sub['rank'].mean()) * len(sub)
                adj_list.append(adjust)
                name_list.append(included_ind)
            adj_list = [i/np.max(adj_list) * 20 for i in adj_list]
            members_industry = pd.DataFrame(list(zip(name_list,adj_list))).set_index(0)[1] + mid_members_industry
            members_industry = members_industry[members_industry>0]
            members_industry = members_industry/members_industry.sum()
            members_industry = round(members_industry.sort_values(ascending=False)*buy_num)
            res = pd.concat([industry_ind.loc[origin]['ind'].value_counts()/buy_num * 100,\
                       members_industry/members_industry.sum() * 100,\
                       mid_members_industry],axis=1)
            res.columns = ['origin','adjusted','hs300']
            res.fillna(0,inplace=True)
            res['result'] = (res['origin'] - res['adjusted']) * 1/2 + res['adjusted']
            members_industry = round(res['result']/res['result'].sum() * buy_num)
            portfolio = []
            for indname in members_industry.index:
                add = ranked_with_industry[(ranked_with_industry['ind'] == indname) &\
                    (ranked_with_industry.index.isin(keeping)) &\
                    (~ranked_with_industry.index.isin(today_prest_kick))].\
                    index.tolist()[:int(members_industry.xs(indname))]
                portfolio += add
            portfolio_origin = []
            for indname in members_industry.index:
                add = ranked_with_industry[(ranked_with_industry['ind'] == indname)].\
                    index.tolist()[:int(members_industry.xs(indname))]
                portfolio_origin += add
            p_data = adjusted_weight_coef.xs(changing_model_date_yesterday).loc[portfolio]
            portfolio_weight = p_data/(p_data.sum())
            portfolio_weight = (portfolio_weight - (portfolio_weight - portfolio_weight.mean())/2).sort_values()
            df = industry_ind.loc[portfolio]
            df['weight'] = portfolio_weight
            df['indcode'] = industry_ind.loc[df.index]
            df['name'] = code_to_name.loc[df.index]
            df.reset_index(inplace=True)
            df = df.loc[:,['name','index','ind','indcode','weight']]
            df.columns = ['股票名称','股票代码','所属行业','行业代码','资金占比']
            txt_df = pd.DataFrame()
            txt_df['证券代码'] = ['0'*(6-len(str(i)))+str(i) for i in df['股票代码']]
            txt_df['证券名称'] = df['股票名称']
            txt_df['每股份数'] = 100.00
            txt_df['金额比例'] = [round(i*100,4) for i in df['资金占比']]
            txt_df[''] = ''
            print('共计：',df['股票代码'].isin(keeping).sum(),'   受影响：',([stock for stock in portfolio_origin if stock not in portfolio]),\
                      '   替换为：',([stock for stock in portfolio if stock not in portfolio_origin]))
            if special_flag == 1:
                generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(changing_model_date)+'_HS300增强_'+str(buy_num)+'（'+account_name+'）'+'.txt'
                print(dealer)
            else:
                generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(changing_model_date)+'_HS300增强_'+str(buy_num)+'.txt'
            print(generated_str)
        else:
            special = ''
            buy_num = int(single_account_str.split('支')[0].split('_')[-1])
            amt_limitation = 50000000
            if out:
                ranked_stock_universe = pd.Series(full_cont[model].xs(changing_model_date))[
                                            (~pd.Series(full_cont[model].xs(changing_model_date)).isin(STcode)) & \
                                            (pd.Series(full_cont[model].xs(changing_model_date)).isin(keeping))].tolist()
            else:
                if amt_limitation != 0:
                    yester_amt = amt.xs(changing_model_date_yesterday).fillna(0)
                    yester_amt = yester_amt[~(yester_amt>amt_limitation)].index.tolist()
                else:
                    yester_amt = []
                if account_code in ['HQHX1',] and int_today >= 20190718:
                    print('HQHX1')
                    ranked_stock_universe = pd.Series(full_cont[model].xs(changing_model_date))[\
                                            (~pd.Series(full_cont[model].xs(changing_model_date)).isin(STcode)) & \
                                            (~pd.Series(full_cont[model].xs(changing_model_date)).isin(yester_amt)) & \
                                            (~pd.Series(full_cont[model].xs(changing_model_date)).isin(today_prest_kick)) & \
                                            (pd.Series(full_cont[model].xs(changing_model_date)).isin(keeping)) & \
                                            (~pd.Series(full_cont[model].xs(changing_model_date)).isin([18,601258,600255,600518,587,\
                                                        603101,600172,600221,600515,600555,600568,600856,506,603619,\
                                                        600525,600119,835,981,600069,600715,662,600112,760,600651,600086,2606,408,\
                                                        600083,600179,806,752,600891,600462,820,600687,600615,600747,982,600485,2477,571,\
                                                        600634,971,600701,600240,939,48,600614,600666,600807,600289]))].tolist()
                else:
                    ranked_stock_universe = pd.Series(full_cont[model].xs(changing_model_date))[\
                                            (~pd.Series(full_cont[model].xs(changing_model_date)).isin(STcode)) & \
                                            (~pd.Series(full_cont[model].xs(changing_model_date)).isin(yester_amt)) & \
                                            (~pd.Series(full_cont[model].xs(changing_model_date)).isin(today_prest_kick)) & \
                                            (pd.Series(full_cont[model].xs(changing_model_date)).isin(keeping))].tolist()
            portfolio = ranked_stock_universe[:buy_num]
            portfolio_weight = pd.Series([1/len(portfolio)]*len(portfolio),index=portfolio)
            df = industry_ind.loc[portfolio]
            df['weight'] = portfolio_weight
            df['indcode'] = industry_ind.loc[df.index]
            df['name'] = code_to_name.loc[df.index]
            df.reset_index(inplace=True)
            df = df.loc[:,['name','index','ind','indcode','weight']]
            df.columns = ['股票名称','股票代码','所属行业','行业代码','资金占比']
            txt_df = pd.DataFrame()
            txt_df['证券代码'] = ['0'*(6-len(str(i)))+str(i) for i in df['股票代码']]
            txt_df['证券名称'] = df['股票名称']
            txt_df['每股份数'] = 100.00
            txt_df['金额比例'] = [round(i*100,4) for i in df['资金占比']]
            txt_df[''] = ''
            if special_flag == 1:
                if amt_limitation == 0:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支（原汁原味）'+'_'+str(changing_model_date)+'（'+account_name+'）'+'.txt'
                else:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支'+'_'+str(changing_model_date)+'（'+account_name+'）'+'.txt'
                print(dealer)
            else:
                if amt_limitation == 0:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支（原汁原味）'+'_'+str(changing_model_date)+'.txt'
                else:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支'+'_'+str(changing_model_date)+'.txt'
            print(generated_str)
        model = single_account_str.split('(')[-1].split('_')[0]
        #txt_df.to_csv(generated_str, encoding = 'gbk', index=False)
        txt_df['产品标识'] = account_code
        model_capital = changing_model_capital
        txt_df['任务资金'] = round(txt_df['金额比例']/100 * model_capital)
        txt_df['模型'] = changing_model_marker
        full_txt_df = pd.concat([full_txt_df, txt_df])
        print(type(model_capital), model_capital)
        print()

In [ ]:
full_txt_df['证券代码'] = full_txt_df['证券代码'].astype(int)
full_txt_df['任务资金'] = full_txt_df['任务资金'].astype(int)
full_txt_df['任务股数'] = np.nan
full_txt_df['方向'] = '多'

In [ ]:
#2日要买的部分

In [ ]:
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')

In [ ]:
special_treatment = np.array(pd.read_excel('两日模型要买资金.xlsx')[pd.read_excel('两日模型要买资金.xlsx')['买入资金']!=0]).tolist()

In [ ]:
special_treatment

In [ ]:
if use_excel:
    print('in track')
    os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')
    full_cont['K4'].loc[int_today] = pd.read_csv('prospect_K4.csv',header = None).iloc[:,-1].tolist()
    #full_cont['K5'].loc[int_today] = pd.read_csv('prospect_K5.csv',header = None).iloc[:,-1].tolist()
    full_cont['K6'].loc[int_today] = pd.read_csv('prospect_K6.csv',header = None).iloc[:,-1].tolist()
    full_cont['K7'].loc[int_today] = pd.read_csv('prospect_K7.csv',header = None).iloc[:,-1].tolist()

In [ ]:
full_special_treatment = pd.DataFrame()
full_special_treatment_personal = pd.DataFrame()
for special_treatment_portfolio_info in special_treatment:
    account_code = special_treatment_portfolio_info[0]
    account_name = special_treatment_portfolio_info[1]
    account_str = special_treatment_portfolio_info[2].split('+')
    special_treatment_date = int_today
    special_treatment_date_yesterday = int_yesterday
    special_treatment_capital = special_treatment_portfolio_info[3]
    if account_code.replace('TH','HQ') in special_treat:
        special_flag = 1
        if account_name in ['金湖宏观量化一期','金湖一号','金湖AI对冲十三期','德聚仁和三号','红墙泰和对冲六号','对冲二十五期']:
            keeping = subhaitong
            dealer = '海通'
        elif account_name in ['于微','金湖AI对冲十八期']:
            keeping = subhuafu
            dealer = '华福'
        elif account_name in ['金湖AI对冲五十期','金湖AI对冲二十二期','九期龙哥','金湖增强四期','金湖AI对冲十期','无量稳健六号',\
                              '私享五号信用','金湖中性对冲六号','红墙泰和指数增强七期']:
            keeping = subyinhe
            dealer = '银河'
        elif account_name in ['']:
            keeping = subguoxin
            dealer = '国信'
        elif account_name in ['金湖AI对冲五十七期']:
            keeping = subguosheng
            dealer = '国盛'
        elif account_name in ['圭源金湖启航','金湖聚乐盈1号','无量稳健二号','无量稳健二号分仓']:
            keeping = subhuatai
            dealer = '华泰'
        elif account_name in ['圭源腾飞10号信用']:
            keeping = subpingan
            dealer = '平安'
        elif account_name in ['金湖AI对冲四十五期','金湖AI对冲四十五期分仓']:
            keeping = sublianxun
            dealer = '联讯'
        elif account_name in ['私享八号']:
            keeping = subcaitong
            dealer = '财通'
        elif account_name in ['金湖AI对冲十五期']:
            keeping = subzhongxin
            dealer = '中信'
        else:
            print('#######################################################################################')
    else:
        special_flag = 0
        keeping = all_stocks
        dealer = '非信用'
    if int_today in prest_kick.index.tolist():
        today_prest_kick = prest_kick.xs(int_today).dropna().tolist()
    else:
        today_prest_kick = []
    print(today_prest_kick)
    ###
    STcode = [int(stock) for stock in STcode_by_date.xs(int_yesterday).dropna().tolist()]
    print(account_name)
    for single_account_str in account_str:
        print(today_prest_kick)
        model = single_account_str.split('(')[-1].split('_')[0]
        universe = preset_hold_days(model, return_universe = 'long')[1]
        if '市值' in single_account_str:
            divider = 2; multiplier = 0
            special = '市值'
            first_n = int(single_account_str.split('支')[0].split('_')[-2])
            ind_num = int(single_account_str.split('支')[0].split('_')[-3])
            ranked_stock_universe = pd.Series(full_cont[model].xs(special_treatment_date))[\
                                        (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(STcode))].tolist()
            yester_mkt_cap = mkt_cap_ard.xs(special_treatment_date_yesterday).loc[ranked_stock_universe]
            def aa(df):
                def standardize(ts):
                    ts = ts.copy().dropna()
                    return (ts - ts.mean()) / ts.std()
                return(standardize(df.iloc[:,1]).sort_values(ascending=False))
            ranked_with_industry_ind = industry_ind.loc[ranked_stock_universe]
            standardized_yester_mkt_cap = pd.concat([ranked_with_industry_ind,yester_mkt_cap],axis=1).groupby('ind').apply(aa)
            standardized_yester_mkt_cap.fillna(0,inplace=True)
            ranked_with_industry_ind['rank'] = range(len(ranked_with_industry_ind))
            portfolio = []
            #for ind in ranked_with_industry_ind.groupby('ind').mean().sort_values('rank').index.tolist()[:first_n]:
            for ind in ((ranked_with_industry_ind.drop_duplicates(subset = ['ind'], keep = 'first').set_index('ind')*multiplier + \
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').mean())['rank']).loc[\
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').count()['rank'][\
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').count()['rank']>=ind_num
                ].index.tolist()].sort_values().index.tolist()[:first_n]:
                if account_code in ['HQHX1',] and int_today >= 20190718:
                    portfolio += ranked_with_industry_ind[(ranked_with_industry_ind.index.isin(keeping))&\
                        (ranked_with_industry_ind['ind'] == ind)&\
                        (~ranked_with_industry_ind.index.isin(today_prest_kick))&\
                        (~ranked_with_industry_ind.index.isin([18,601258,600255,600518,587,\
                        603101,600172,600221,600515,600555,600568,600856,506,603619,\
                        600525,600119,835,981,600069,600715,662,600112,760,600651,600086,2606,408,\
                        600083,600179,806,752,600891,600462,820,600687,600615,600747,982,600485,2477,571,\
                        600634,971,600701,600240,939,48,600614,600666,600807,600289]))].iloc[:ind_num].index.tolist()
                else:
                    portfolio += ranked_with_industry_ind[(ranked_with_industry_ind.index.isin(keeping))&\
                        (ranked_with_industry_ind['ind'] == ind)&\
                        (~ranked_with_industry_ind.index.isin(today_prest_kick))].iloc[:ind_num].index.tolist()
            portfolio_origin = []
            #for ind in ranked_with_industry_ind.groupby('ind').mean().sort_values('rank').index.tolist()[:first_n]:
            for ind in ((ranked_with_industry_ind.drop_duplicates(subset = ['ind'], keep = 'first').set_index('ind')*multiplier + \
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').mean())['rank']).loc[\
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').count()['rank'][\
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').count()['rank']>=ind_num
                ].index.tolist()].sort_values().index.tolist()[:first_n]:
                portfolio_origin += ranked_with_industry_ind[(ranked_with_industry_ind['ind'] == ind)].iloc[:ind_num].index.tolist()
            standardized_yester_mkt_cap = standardized_yester_mkt_cap.reset_index().set_index('level_1').loc[portfolio]
            standardized_yester_mkt_cap['quantile'] = scs.norm.cdf(standardized_yester_mkt_cap.reset_index().set_index('level_1').\
                loc[portfolio].iloc[:,1])
            portfolio_weight = pd.Series([stock_weight/standardized_yester_mkt_cap['quantile'].sum() \
                for stock_weight in standardized_yester_mkt_cap['quantile']],index=portfolio)
            while(True):
                if portfolio_weight.max() / portfolio_weight.min() <= 3:
                    break
                else:
                    portfolio_weight = portfolio_weight - (portfolio_weight - portfolio_weight.mean())/2
            df = industry_ind.loc[portfolio]
            df['weight'] = portfolio_weight
            df['indcode'] = industry_ind.loc[df.index]
            df['name'] = code_to_name.loc[df.index]
            df.reset_index(inplace=True)
            df = df.loc[:,['name','index','ind','indcode','weight']]
            df.columns = ['股票名称','股票代码','所属行业','行业代码','资金占比']
            txt_df = pd.DataFrame()
            txt_df['股票代码'] = ['0'*(6-len(str(i)))+str(i) for i in df['股票代码']]
            txt_df['股票名称'] = df['股票名称']
            txt_df['每股份数'] = 100.00
            txt_df['金额比例'] = [round(i*100,4) for i in df['资金占比']]
            txt_df[''] = ''
            print('共计：',df['股票代码'].isin(keeping).sum(),'   受影响：',([stock for stock in portfolio_origin if stock not in portfolio]),\
                      '   替换为：',([stock for stock in portfolio if stock not in portfolio_origin]))
            if special_flag == 1:
                generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(special_treatment_date)+'_市值行业_'+str(ind_num)+'_'+str(first_n)+'（'+account_name+'）'+'.txt'
                print(dealer)
            else:
                generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(special_treatment_date)+'_市值行业_'+str(ind_num)+'_'+str(first_n)+'.txt'
            print(generated_str)
        elif '原味' in single_account_str:
            special = '原味'
            buy_num = int(single_account_str.split('支')[0].split('_')[-1])
            amt_limitation = 0
            if amt_limitation != 0:
                yester_amt = amt.xs(special_treatment_date_yesterday).fillna(0)
                yester_amt = yester_amt[~(yester_amt>amt_limitation)].index.tolist()
            else:
                yester_amt = []
            if account_code in ['HQHX1',] and int_today >= 20190718:
                ranked_stock_universe = pd.Series(full_cont[model].xs(int_today))[\
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(STcode)) & \
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(yester_amt)) & \
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(today_prest_kick)) & \
                                        (pd.Series(full_cont[model].xs(int_today)).isin(keeping)) & \
                                        (~pd.Series(full_cont[model].xs(int_today)).isin([18,601258,600255,600518,587,\
                                                    603101,600172,600221,600515,600555,600568,600856,506,603619,\
                                                    600525,600119,835,981,600069,600715,662,600112,760,600651,600086,2606,408,\
                                                    600083,600179,806,752,600891,600462,820,600687,600615,600747,982,600485,2477,571,\
                                                    600634,971,600701,600240,939,48,600614,600666,600807,600289]))].tolist()
            else:
                ranked_stock_universe = pd.Series(full_cont[model].xs(int_today))[\
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(STcode)) & \
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(yester_amt)) & \
                                        (~pd.Series(full_cont[model].xs(int_today)).isin(today_prest_kick)) & \
                                        (pd.Series(full_cont[model].xs(int_today)).isin(keeping))].tolist()
            portfolio = ranked_stock_universe[:buy_num]
            ranked_stock_universe_origin = [st for st in full_cont[model].xs(special_treatment_date) if st not in STcode \
                                            and st not in yester_amt]
            portfolio_origin = ranked_stock_universe_origin[:buy_num]
            portfolio_weight = pd.Series([1/len(portfolio)]*len(portfolio),index=portfolio)
            df = industry_ind.loc[portfolio]
            df['weight'] = portfolio_weight
            df['indcode'] = industry_ind.loc[df.index]
            df['name'] = code_to_name.loc[df.index]
            df.reset_index(inplace=True)
            df = df.loc[:,['name','index','ind','indcode','weight']]
            df.columns = ['股票名称','股票代码','所属行业','行业代码','资金占比']
            txt_df = pd.DataFrame()
            txt_df['股票代码'] = ['0'*(6-len(str(i)))+str(i) for i in df['股票代码']]
            txt_df['股票名称'] = df['股票名称']
            txt_df['每股份数'] = 100.00
            txt_df['金额比例'] = [round(i*100,4) for i in df['资金占比']]
            txt_df[''] = ''
            print('共计：',df['股票代码'].isin(keeping).sum(),'   受影响：',([stock for stock in portfolio_origin if stock not in portfolio]),\
                      '   替换为：',([stock for stock in portfolio if stock not in portfolio_origin]))
            if special_flag == 1:
                if amt_limitation == 0:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支（原汁原味）'+'_'+str(special_treatment_date)+'（'+account_name+'）'+'.txt'
                else:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支'+'_'+str(special_treatment_date)+'（'+account_name+'）'+'.txt'
                print(dealer)
            else:
                if amt_limitation == 0:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支（原汁原味）'+'_'+str(special_treatment_date)+'.txt'
                else:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支'+'_'+str(special_treatment_date)+'.txt'
            print(generated_str)
        elif '沪深' in single_account_str:
            special = '沪深'
            buy_num = int(single_account_str.split('支')[0].split('_')[-1])
            amt_limitation = 50000000
            if amt_limitation != 0:
                yester_amt = amt.xs(special_treatment_date_yesterday).fillna(0)
                yester_amt = yester_amt[~(yester_amt>amt_limitation)].index.tolist()
            else:
                yester_amt = []
            ranked_stock_universe = pd.Series(full_cont[model].xs(special_treatment_date))[\
                                    (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(STcode)) & \
                                    (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(yester_amt))].tolist()
            origin = ranked_stock_universe[:buy_num]
            #HS300行业权重
            members = hs300members.xs(special_treatment_date_yesterday).tolist()
            mid_members_industry = (pd.concat([(\
                adjusted_weight_coef.xs(special_treatment_date_yesterday).loc[members]/\
                adjusted_weight_coef.xs(special_treatment_date_yesterday).loc[members]\
                .sum()).sort_values(),industry_ind['ind'].loc[members]],axis=1).groupby('ind').sum().iloc[:,0]*100)\
                .sort_values(ascending=False)
            #ranked_stock_universe个股行业
            ranked_with_industry = industry_ind.loc[ranked_stock_universe].dropna()
            ranked_with_industry['rrank'] = range(len(ranked_with_industry))
            exp = ranked_with_industry.sort_values('rrank').dropna()
            exp['rank'] = range(1,len(exp)+1)
            adj_list = []
            name_list = []
            for included_ind in exp['ind'].unique():
                sub = exp[exp['ind'] == included_ind]
                adjust = (exp['rank'].mean() - sub['rank'].mean()) * len(sub)
                adj_list.append(adjust)
                name_list.append(included_ind)
            adj_list = [i/np.max(adj_list) * 20 for i in adj_list]
            members_industry = pd.DataFrame(list(zip(name_list,adj_list))).set_index(0)[1] + mid_members_industry
            members_industry = members_industry[members_industry>0]
            members_industry = members_industry/members_industry.sum()
            members_industry = round(members_industry.sort_values(ascending=False)*buy_num)
            res = pd.concat([industry_ind.loc[origin]['ind'].value_counts()/buy_num * 100,\
                       members_industry/members_industry.sum() * 100,\
                       mid_members_industry],axis=1)
            res.columns = ['origin','adjusted','hs300']
            res.fillna(0,inplace=True)
            res['result'] = (res['origin'] - res['adjusted']) * 1/2 + res['adjusted']
            members_industry = round(res['result']/res['result'].sum() * buy_num)
            portfolio = []
            for indname in members_industry.index:
                add = ranked_with_industry[(ranked_with_industry['ind'] == indname) &\
                    (ranked_with_industry.index.isin(keeping)) &\
                    (~ranked_with_industry.index.isin(today_prest_kick))].\
                    index.tolist()[:int(members_industry.xs(indname))]
                portfolio += add
            portfolio_origin = []
            for indname in members_industry.index:
                add = ranked_with_industry[(ranked_with_industry['ind'] == indname)].\
                    index.tolist()[:int(members_industry.xs(indname))]
                portfolio_origin += add
            p_data = adjusted_weight_coef.xs(special_treatment_date_yesterday).loc[portfolio]
            portfolio_weight = p_data/(p_data.sum())
            portfolio_weight = (portfolio_weight - (portfolio_weight - portfolio_weight.mean())/2).sort_values()
            df = industry_ind.loc[portfolio]
            df['weight'] = portfolio_weight
            df['indcode'] = industry_ind.loc[df.index]
            df['name'] = code_to_name.loc[df.index]
            df.reset_index(inplace=True)
            df = df.loc[:,['name','index','ind','indcode','weight']]
            df.columns = ['股票名称','股票代码','所属行业','行业代码','资金占比']
            txt_df = pd.DataFrame()
            txt_df['股票代码'] = ['0'*(6-len(str(i)))+str(i) for i in df['股票代码']]
            txt_df['股票名称'] = df['股票名称']
            txt_df['每股份数'] = 100.00
            txt_df['金额比例'] = [round(i*100,4) for i in df['资金占比']]
            txt_df[''] = ''
            print('共计：',df['股票代码'].isin(keeping).sum(),'   受影响：',([stock for stock in portfolio_origin if stock not in portfolio]),\
                      '   替换为：',([stock for stock in portfolio if stock not in portfolio_origin]))
            if special_flag == 1:
                generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(special_treatment_date)+'_HS300增强_'+str(buy_num)+'（'+account_name+'）'+'.txt'
                print(dealer)
            else:
                generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(special_treatment_date)+'_HS300增强_'+str(buy_num)+'.txt'
            print(generated_str)
        else:
            special = ''
            buy_num = int(single_account_str.split('支')[0].split('_')[-1])
            amt_limitation = 50000000
            if out:
                ranked_stock_universe = pd.Series(full_cont[model].xs(int_today))[
                                            (~pd.Series(full_cont[model].xs(int_today)).isin(STcode)) & \
                                            (pd.Series(full_cont[model].xs(int_today)).isin(keeping))].tolist()
            else:
                if amt_limitation != 0:
                    yester_amt = amt.xs(special_treatment_date_yesterday).fillna(0)
                    yester_amt = yester_amt[~(yester_amt>amt_limitation)].index.tolist()
                else:
                    yester_amt = []
                if account_code in ['HQHX1',] and int_today >= 20190718:
                    ranked_stock_universe = pd.Series(full_cont[model].xs(int_today))[\
                                            (~pd.Series(full_cont[model].xs(int_today)).isin(STcode)) & \
                                            (~pd.Series(full_cont[model].xs(int_today)).isin(yester_amt)) & \
                                            (~pd.Series(full_cont[model].xs(int_today)).isin(today_prest_kick)) & \
                                            (pd.Series(full_cont[model].xs(int_today)).isin(keeping)) & \
                                            (~pd.Series(full_cont[model].xs(int_today)).isin([18,601258,600255,600518,587,\
                                                        603101,600172,600221,600515,600555,600568,600856,506,603619,\
                                                        600525,600119,835,981,600069,600715,662,600112,760,600651,600086,2606,408,\
                                                        600083,600179,806,752,600891,600462,820,600687,600615,600747,982,600485,2477,571,\
                                                        600634,971,600701,600240,939,48,600614,600666,600807,600289]))].tolist()
                else:
                    ranked_stock_universe = pd.Series(full_cont[model].xs(int_today))[\
                                            (~pd.Series(full_cont[model].xs(int_today)).isin(STcode)) & \
                                            (~pd.Series(full_cont[model].xs(int_today)).isin(yester_amt)) & \
                                            (~pd.Series(full_cont[model].xs(int_today)).isin(today_prest_kick)) & \
                                            (pd.Series(full_cont[model].xs(int_today)).isin(keeping))].tolist()
            portfolio = ranked_stock_universe[:buy_num]
            portfolio_weight = pd.Series([1/len(portfolio)]*len(portfolio),index=portfolio)
            df = industry_ind.loc[portfolio]
            df['weight'] = portfolio_weight
            df['indcode'] = industry_ind.loc[df.index]
            df['name'] = code_to_name.loc[df.index]
            df.reset_index(inplace=True)
            df = df.loc[:,['name','index','ind','indcode','weight']]
            df.columns = ['股票名称','股票代码','所属行业','行业代码','资金占比']
            txt_df = pd.DataFrame()
            txt_df['股票代码'] = ['0'*(6-len(str(i)))+str(i) for i in df['股票代码']]
            txt_df['股票名称'] = df['股票名称']
            txt_df['每股份数'] = 100.00
            txt_df['金额比例'] = [round(i*100,4) for i in df['资金占比']]
            txt_df[''] = ''
        model = single_account_str.split('(')[-1].split('_')[0]
        model_capital = special_treatment_capital
        txt_df['任务资金'] = round(txt_df['金额比例']/100 * model_capital)
        txt_df['昨收'] = close.iloc[-1].loc[txt_df['股票代码'].astype(int)].tolist()
        txt_df['总股数'] = ((txt_df['任务资金']/txt_df['昨收'])//100*100).tolist()
        txt_df.dropna(inplace = True)
        personal_txt_df = txt_df[['股票代码','股票名称','任务资金','总股数']]
        txt_df = txt_df[['股票代码','股票名称','总股数']]
        txt_df['总股数'] = txt_df['总股数'].astype(int)
        txt_df['产品标识'] = account_code
        personal_txt_df['模型'] = model
        personal_txt_df['产品标识'] = account_code
        full_special_treatment = pd.concat([full_special_treatment, txt_df])
        full_special_treatment_personal = pd.concat([full_special_treatment_personal, personal_txt_df])
        print(type(model_capital), model_capital)
        print()
full_special_treatment.to_excel('2days.xlsx', index = False, encoding = 'gbk')
full_special_treatment_personal.to_excel(str(int_today)+'_2days.xlsx', index = False, encoding = 'gbk')

In [ ]:
full = full_special_treatment_personal.groupby(['产品标识','股票代码']).sum().reset_index()
full['股票名称'] = code_to_name.loc[full['股票代码'].astype(int)].tolist()

for account_code in full['产品标识'].unique().tolist():
    account_name = pd.read_excel('两日模型要买资金.xlsx')[pd.read_excel('两日模型要买资金.xlsx')['代码'] == account_code].iloc[0,1]
    sub = full[full['产品标识'] == account_code]
    sub[''] = ''
    sub = sub[['股票代码','股票名称','总股数','任务资金','',]]
    sub[['总股数','任务资金',]] = sub[['总股数','任务资金',]].astype(int)
    sub['任务资金'] = round(sub['任务资金']/sub['任务资金'].sum()*100,4)
    sub.columns = [['证券代码','证券名称','总股数','金额比例','']]
    sub.to_csv(account_code+account_name+' '+str(int_today)+'.txt',index=False,encoding='gbk')

for account_code in full['产品标识'].unique().tolist():
    account_name = pd.read_excel('两日模型要买资金.xlsx')[pd.read_excel('两日模型要买资金.xlsx')['代码'] == account_code].iloc[0,1]
    sub = full[full['产品标识'] == account_code]
    sub[''] = ''
    sub = sub[['股票代码','股票名称','总股数','任务资金','',]]
    sub[['总股数','任务资金',]] = sub[['总股数','任务资金',]].astype(int)
    sub['任务资金'] = sub['任务资金']/sub['任务资金'].sum()*100
    sub.columns = ['证券代码','证券名称','每股份数','金额比例','']
    sub.to_excel(account_code+account_name+' '+str(int_today)+'_s.xlsx',index=False,encoding='gbk')

In [ ]:
for account_code in full['产品标识'].unique().tolist():
    account_name = pd.read_excel('两日模型要买资金.xlsx')[pd.read_excel('两日模型要买资金.xlsx')['代码'] == account_code].iloc[0,1]
    sub = full[full['产品标识'] == account_code]
    sub[''] = ''
    sub = sub[['股票代码','股票名称','任务资金','总股数',]]
    sub[['任务资金','总股数']] = sub[['任务资金','总股数']].astype(int)
    sub['成交数量'] = sub['总股数']
    sub.columns = ['证券代码','证券名称','任务资金','委托数量','成交数量']
    sub.to_excel(account_code+account_name+' '+str(int_today)+'委托单.xlsx',index=False,encoding='gbk')

In [ ]:
full_special_treatment_buy = full_special_treatment_personal.copy()
full_special_treatment_buy['任务股数'] = np.nan
full_special_treatment_buy['方向'] = '多'
full_special_treatment_buy = full_special_treatment_buy[['产品标识','股票代码','任务资金','任务股数','方向']]

In [ ]:
full_special_treatment_buy.to_excel('总任务管理表_买入_2日.xlsx', index = False, encoding = 'gbk')

In [ ]:
pd.concat([full_special_treatment_buy, pd.read_excel('总任务管理表_卖出_2日.xlsx')]).to_excel('总任务管理表_2日.xlsx', index = False, encoding = 'gbk')

In [ ]:
#建仓、补仓要买的部分

In [ ]:
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')

In [ ]:
special_treatment = np.array(pd.read_excel('手动补仓模型要买资金.xlsx')[pd.read_excel('手动补仓模型要买资金.xlsx')['买入资金']!=0]).tolist()

In [ ]:
special_treatment

In [ ]:
full_special_treatment = pd.DataFrame()
full_special_treatment_personal = pd.DataFrame()
for special_treatment_portfolio_info in special_treatment:
    account_code = special_treatment_portfolio_info[0]
    account_name = special_treatment_portfolio_info[1]
    account_str = special_treatment_portfolio_info[2].split('+')
    special_treatment_date = special_treatment_portfolio_info[4]
    special_treatment_date_yesterday = int(get_n_week_date_before(str(special_treatment_date)))
    special_treatment_capital = special_treatment_portfolio_info[3]
    if account_code.replace('RB','HQ') in special_treat:
        special_flag = 1
        if account_name in ['金湖宏观量化一期','金湖一号','金湖AI对冲十三期','德聚仁和三号','红墙泰和对冲六号','对冲二十五期']:
            keeping = subhaitong
            dealer = '海通'
        elif account_name in ['于微','金湖AI对冲十八期']:
            keeping = subhuafu
            dealer = '华福'
        elif account_name in ['金湖AI对冲五十期','金湖AI对冲二十二期','九期龙哥','金湖增强四期','金湖AI对冲十期','无量稳健六号',\
                              '私享五号信用','金湖中性对冲六号','红墙泰和指数增强七期']:
            keeping = subyinhe
            dealer = '银河'
        elif account_name in ['']:
            keeping = subguoxin
            dealer = '国信'
        elif account_name in ['金湖AI对冲五十七期']:
            keeping = subguosheng
            dealer = '国盛'
        elif account_name in ['圭源金湖启航','金湖聚乐盈1号','无量稳健二号','无量稳健二号分仓']:
            keeping = subhuatai
            dealer = '华泰'
        elif account_name in ['圭源腾飞10号信用']:
            keeping = subpingan
            dealer = '平安'
        elif account_name in ['金湖AI对冲四十五期','金湖AI对冲四十五期分仓']:
            keeping = sublianxun
            dealer = '联讯'
        elif account_name in ['私享八号']:
            keeping = subcaitong
            dealer = '财通'
        elif account_name in ['金湖AI对冲十五期']:
            keeping = subzhongxin
            dealer = '中信'
        else:
            print('#######################################################################################')
        print(dealer)
    else:
        special_flag = 0
        keeping = all_stocks
        dealer = '非信用'
    if int_today in prest_kick.index.tolist():
        today_prest_kick = prest_kick.xs(int_today).dropna().tolist()
    else:
        today_prest_kick = []
    print(today_prest_kick)
    ###
    STcode = [int(stock) for stock in STcode_by_date.xs(int_yesterday).dropna().tolist()]
    print(account_name)
    for single_account_str in account_str:
        print(today_prest_kick)
        model = single_account_str.split('(')[-1].split('_')[0]
        universe = preset_hold_days(model, return_universe = 'long')[1]
        if '市值' in single_account_str:
            divider = 2; multiplier = 0
            special = '市值'
            first_n = int(single_account_str.split('支')[0].split('_')[-2])
            ind_num = int(single_account_str.split('支')[0].split('_')[-3])
            ranked_stock_universe = pd.Series(full_cont[model].xs(special_treatment_date))[\
                                        (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(STcode))].tolist()
            yester_mkt_cap = mkt_cap_ard.xs(special_treatment_date_yesterday).loc[ranked_stock_universe]
            def aa(df):
                def standardize(ts):
                    ts = ts.copy().dropna()
                    return (ts - ts.mean()) / ts.std()
                return(standardize(df.iloc[:,1]).sort_values(ascending=False))
            ranked_with_industry_ind = industry_ind.loc[ranked_stock_universe]
            standardized_yester_mkt_cap = pd.concat([ranked_with_industry_ind,yester_mkt_cap],axis=1).groupby('ind').apply(aa)
            standardized_yester_mkt_cap.fillna(0,inplace=True)
            ranked_with_industry_ind['rank'] = range(len(ranked_with_industry_ind))
            portfolio = []
            #for ind in ranked_with_industry_ind.groupby('ind').mean().sort_values('rank').index.tolist()[:first_n]:
            for ind in ((ranked_with_industry_ind.drop_duplicates(subset = ['ind'], keep = 'first').set_index('ind')*multiplier + \
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').mean())['rank']).loc[\
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').count()['rank'][\
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').count()['rank']>=ind_num
                ].index.tolist()].sort_values().index.tolist()[:first_n]:
                if account_code in ['HQHX1',] and special_treatment_date >= 20190718:
                    portfolio += ranked_with_industry_ind[(ranked_with_industry_ind.index.isin(keeping))&\
                        (ranked_with_industry_ind['ind'] == ind)&\
                        (~ranked_with_industry_ind.index.isin(today_prest_kick))&\
                        (~ranked_with_industry_ind.index.isin([18,601258,600255,600518,587,\
                        603101,600172,600221,600515,600555,600568,600856,506,603619,\
                        600525,600119,835,981,600069,600715,662,600112,760,600651,600086,2606,408,\
                        600083,600179,806,752,600891,600462,820,600687,600615,600747,982,600485,2477,571,\
                        600634,971,600701,600240,939,48,600614,600666,600807,600289]))].iloc[:ind_num].index.tolist()
                else:
                    portfolio += ranked_with_industry_ind[(ranked_with_industry_ind.index.isin(keeping))&\
                        (ranked_with_industry_ind['ind'] == ind)&\
                        (~ranked_with_industry_ind.index.isin(today_prest_kick))].iloc[:ind_num].index.tolist()
            portfolio_origin = []
            #for ind in ranked_with_industry_ind.groupby('ind').mean().sort_values('rank').index.tolist()[:first_n]:
            for ind in ((ranked_with_industry_ind.drop_duplicates(subset = ['ind'], keep = 'first').set_index('ind')*multiplier + \
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').mean())['rank']).loc[\
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').count()['rank'][\
                ranked_with_industry_ind.iloc[:int(len(ranked_with_industry_ind)*(1/divider))].groupby('ind').count()['rank']>=ind_num
                ].index.tolist()].sort_values().index.tolist()[:first_n]:
                portfolio_origin += ranked_with_industry_ind[(ranked_with_industry_ind['ind'] == ind)].iloc[:ind_num].index.tolist()
            standardized_yester_mkt_cap = standardized_yester_mkt_cap.reset_index().set_index('level_1').loc[portfolio]
            standardized_yester_mkt_cap['quantile'] = scs.norm.cdf(standardized_yester_mkt_cap.reset_index().set_index('level_1').\
                loc[portfolio].iloc[:,1])
            portfolio_weight = pd.Series([stock_weight/standardized_yester_mkt_cap['quantile'].sum() \
                for stock_weight in standardized_yester_mkt_cap['quantile']],index=portfolio)
            while(True):
                if portfolio_weight.max() / portfolio_weight.min() <= 3:
                    break
                else:
                    portfolio_weight = portfolio_weight - (portfolio_weight - portfolio_weight.mean())/2
            df = industry_ind.loc[portfolio]
            df['weight'] = portfolio_weight
            df['indcode'] = industry_ind.loc[df.index]
            df['name'] = code_to_name.loc[df.index]
            df.reset_index(inplace=True)
            df = df.loc[:,['name','index','ind','indcode','weight']]
            df.columns = ['股票名称','股票代码','所属行业','行业代码','资金占比']
            txt_df = pd.DataFrame()
            txt_df['股票代码'] = ['0'*(6-len(str(i)))+str(i) for i in df['股票代码']]
            txt_df['股票名称'] = df['股票名称']
            txt_df['每股份数'] = 100.00
            txt_df['金额比例'] = [round(i*100,4) for i in df['资金占比']]
            txt_df[''] = ''
            print('共计：',df['股票代码'].isin(keeping).sum(),'   受影响：',([stock for stock in portfolio_origin if stock not in portfolio]),\
                      '   替换为：',([stock for stock in portfolio if stock not in portfolio_origin]))
            if special_flag == 1:
                generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(special_treatment_date)+'_市值行业_'+str(ind_num)+'_'+str(first_n)+'（'+account_name+'）'+'.txt'
                print(dealer)
            else:
                generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(special_treatment_date)+'_市值行业_'+str(ind_num)+'_'+str(first_n)+'.txt'
            print(generated_str)
        elif '原味' in single_account_str:
            special = '原味'
            buy_num = int(single_account_str.split('支')[0].split('_')[-1])
            amt_limitation = 0
            if amt_limitation != 0:
                yester_amt = amt.xs(special_treatment_date_yesterday).fillna(0)
                yester_amt = yester_amt[~(yester_amt>amt_limitation)].index.tolist()
            else:
                yester_amt = []
            if account_code in ['HQHX1',] and special_treatment_date >= 20190718:
                ranked_stock_universe = pd.Series(full_cont[model].xs(special_treatment_date))[\
                                        (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(STcode)) & \
                                        (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(yester_amt)) & \
                                        (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(today_prest_kick)) & \
                                        (pd.Series(full_cont[model].xs(special_treatment_date)).isin(keeping)) & \
                                        (~pd.Series(full_cont[model].xs(special_treatment_date)).isin([18,601258,600255,600518,587,\
                                                    603101,600172,600221,600515,600555,600568,600856,506,603619,\
                                                    600525,600119,835,981,600069,600715,662,600112,760,600651,600086,2606,408,\
                                                    600083,600179,806,752,600891,600462,820,600687,600615,600747,982,600485,2477,571,\
                                                    600634,971,600701,600240,939,48,600614,600666,600807,600289]))].tolist()
            else:
                ranked_stock_universe = pd.Series(full_cont[model].xs(special_treatment_date))[\
                                        (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(STcode)) & \
                                        (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(yester_amt)) & \
                                        (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(today_prest_kick)) & \
                                        (pd.Series(full_cont[model].xs(special_treatment_date)).isin(keeping))].tolist()
            portfolio = ranked_stock_universe[:buy_num]
            ranked_stock_universe_origin = [st for st in full_cont[model].xs(special_treatment_date) if st not in STcode \
                                            and st not in yester_amt]
            portfolio_origin = ranked_stock_universe_origin[:buy_num]
            portfolio_weight = pd.Series([1/len(portfolio)]*len(portfolio),index=portfolio)
            df = industry_ind.loc[portfolio]
            df['weight'] = portfolio_weight
            df['indcode'] = industry_ind.loc[df.index]
            df['name'] = code_to_name.loc[df.index]
            df.reset_index(inplace=True)
            df = df.loc[:,['name','index','ind','indcode','weight']]
            df.columns = ['股票名称','股票代码','所属行业','行业代码','资金占比']
            txt_df = pd.DataFrame()
            txt_df['股票代码'] = ['0'*(6-len(str(i)))+str(i) for i in df['股票代码']]
            txt_df['股票名称'] = df['股票名称']
            txt_df['每股份数'] = 100.00
            txt_df['金额比例'] = [round(i*100,4) for i in df['资金占比']]
            txt_df[''] = ''
            print('共计：',df['股票代码'].isin(keeping).sum(),'   受影响：',([stock for stock in portfolio_origin if stock not in portfolio]),\
                      '   替换为：',([stock for stock in portfolio if stock not in portfolio_origin]))
            if special_flag == 1:
                if amt_limitation == 0:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支（原汁原味）'+'_'+str(special_treatment_date)+'（'+account_name+'）'+'.txt'
                else:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支'+'_'+str(special_treatment_date)+'（'+account_name+'）'+'.txt'
                print(dealer)
            else:
                if amt_limitation == 0:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支（原汁原味）'+'_'+str(special_treatment_date)+'.txt'
                else:
                    generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(buy_num)+'支'+'_'+str(special_treatment_date)+'.txt'
            print(generated_str)
        elif '沪深' in single_account_str:
            special = '沪深'
            buy_num = int(single_account_str.split('支')[0].split('_')[-1])
            amt_limitation = 50000000
            if amt_limitation != 0:
                yester_amt = amt.xs(special_treatment_date_yesterday).fillna(0)
                yester_amt = yester_amt[~(yester_amt>amt_limitation)].index.tolist()
            else:
                yester_amt = []
            ranked_stock_universe = pd.Series(full_cont[model].xs(special_treatment_date))[\
                                    (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(STcode)) & \
                                    (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(yester_amt))].tolist()
            origin = ranked_stock_universe[:buy_num]
            #HS300行业权重
            members = hs300members.xs(special_treatment_date_yesterday).tolist()
            mid_members_industry = (pd.concat([(\
                adjusted_weight_coef.xs(special_treatment_date_yesterday).loc[members]/\
                adjusted_weight_coef.xs(special_treatment_date_yesterday).loc[members]\
                .sum()).sort_values(),industry_ind['ind'].loc[members]],axis=1).groupby('ind').sum().iloc[:,0]*100)\
                .sort_values(ascending=False)
            #ranked_stock_universe个股行业
            ranked_with_industry = industry_ind.loc[ranked_stock_universe].dropna()
            ranked_with_industry['rrank'] = range(len(ranked_with_industry))
            exp = ranked_with_industry.sort_values('rrank').dropna()
            exp['rank'] = range(1,len(exp)+1)
            adj_list = []
            name_list = []
            for included_ind in exp['ind'].unique():
                sub = exp[exp['ind'] == included_ind]
                adjust = (exp['rank'].mean() - sub['rank'].mean()) * len(sub)
                adj_list.append(adjust)
                name_list.append(included_ind)
            adj_list = [i/np.max(adj_list) * 20 for i in adj_list]
            members_industry = pd.DataFrame(list(zip(name_list,adj_list))).set_index(0)[1] + mid_members_industry
            members_industry = members_industry[members_industry>0]
            members_industry = members_industry/members_industry.sum()
            members_industry = round(members_industry.sort_values(ascending=False)*buy_num)
            res = pd.concat([industry_ind.loc[origin]['ind'].value_counts()/buy_num * 100,\
                       members_industry/members_industry.sum() * 100,\
                       mid_members_industry],axis=1)
            res.columns = ['origin','adjusted','hs300']
            res.fillna(0,inplace=True)
            res['result'] = (res['origin'] - res['adjusted']) * 1/2 + res['adjusted']
            members_industry = round(res['result']/res['result'].sum() * buy_num)
            portfolio = []
            for indname in members_industry.index:
                add = ranked_with_industry[(ranked_with_industry['ind'] == indname) &\
                    (ranked_with_industry.index.isin(keeping)) &\
                    (~ranked_with_industry.index.isin(today_prest_kick))].\
                    index.tolist()[:int(members_industry.xs(indname))]
                portfolio += add
            portfolio_origin = []
            for indname in members_industry.index:
                add = ranked_with_industry[(ranked_with_industry['ind'] == indname)].\
                    index.tolist()[:int(members_industry.xs(indname))]
                portfolio_origin += add
            p_data = adjusted_weight_coef.xs(special_treatment_date_yesterday).loc[portfolio]
            portfolio_weight = p_data/(p_data.sum())
            portfolio_weight = (portfolio_weight - (portfolio_weight - portfolio_weight.mean())/2).sort_values()
            df = industry_ind.loc[portfolio]
            df['weight'] = portfolio_weight
            df['indcode'] = industry_ind.loc[df.index]
            df['name'] = code_to_name.loc[df.index]
            df.reset_index(inplace=True)
            df = df.loc[:,['name','index','ind','indcode','weight']]
            df.columns = ['股票名称','股票代码','所属行业','行业代码','资金占比']
            txt_df = pd.DataFrame()
            txt_df['股票代码'] = ['0'*(6-len(str(i)))+str(i) for i in df['股票代码']]
            txt_df['股票名称'] = df['股票名称']
            txt_df['每股份数'] = 100.00
            txt_df['金额比例'] = [round(i*100,4) for i in df['资金占比']]
            txt_df[''] = ''
            print('共计：',df['股票代码'].isin(keeping).sum(),'   受影响：',([stock for stock in portfolio_origin if stock not in portfolio]),\
                      '   替换为：',([stock for stock in portfolio if stock not in portfolio_origin]))
            if special_flag == 1:
                generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(special_treatment_date)+'_HS300增强_'+str(buy_num)+'（'+account_name+'）'+'.txt'
                print(dealer)
            else:
                generated_str = model+'_'+version_num[model]+'_'+universe+'_'+str(special_treatment_date)+'_HS300增强_'+str(buy_num)+'.txt'
            print(generated_str)
        else:
            special = ''
            buy_num = int(single_account_str.split('支')[0].split('_')[-1])
            amt_limitation = 50000000
            if out:
                ranked_stock_universe = pd.Series(full_cont[model].xs(special_treatment_date))[
                                            (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(STcode)) & \
                                            (pd.Series(full_cont[model].xs(special_treatment_date)).isin(keeping))].tolist()
            else:
                if amt_limitation != 0:
                    yester_amt = amt.xs(special_treatment_date_yesterday).fillna(0)
                    yester_amt = yester_amt[~(yester_amt>amt_limitation)].index.tolist()
                else:
                    yester_amt = []
                if account_code in ['HQHX1',] and special_treatment_date >= 20190718:
                    ranked_stock_universe = pd.Series(full_cont[model].xs(special_treatment_date))[\
                                            (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(STcode)) & \
                                            (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(yester_amt)) & \
                                            (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(today_prest_kick)) & \
                                            (pd.Series(full_cont[model].xs(special_treatment_date)).isin(keeping)) & \
                                            (~pd.Series(full_cont[model].xs(special_treatment_date)).isin([18,601258,600255,600518,587,\
                                                        603101,600172,600221,600515,600555,600568,600856,506,603619,\
                                                        600525,600119,835,981,600069,600715,662,600112,760,600651,600086,2606,408,\
                                                        600083,600179,806,752,600891,600462,820,600687,600615,600747,982,600485,2477,571,\
                                                        600634,971,600701,600240,939,48,600614,600666,600807,600289]))].tolist()
                else:
                    ranked_stock_universe = pd.Series(full_cont[model].xs(special_treatment_date))[\
                                            (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(STcode)) & \
                                            (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(yester_amt)) & \
                                            (~pd.Series(full_cont[model].xs(special_treatment_date)).isin(today_prest_kick)) & \
                                            (pd.Series(full_cont[model].xs(special_treatment_date)).isin(keeping))].tolist()
            portfolio = ranked_stock_universe[:buy_num]
            portfolio_weight = pd.Series([1/len(portfolio)]*len(portfolio),index=portfolio)
            df = industry_ind.loc[portfolio]
            df['weight'] = portfolio_weight
            df['indcode'] = industry_ind.loc[df.index]
            df['name'] = code_to_name.loc[df.index]
            df.reset_index(inplace=True)
            df = df.loc[:,['name','index','ind','indcode','weight']]
            df.columns = ['股票名称','股票代码','所属行业','行业代码','资金占比']
            txt_df = pd.DataFrame()
            txt_df['股票代码'] = ['0'*(6-len(str(i)))+str(i) for i in df['股票代码']]
            txt_df['股票名称'] = df['股票名称']
            txt_df['每股份数'] = 100.00
            txt_df['金额比例'] = [round(i*100,4) for i in df['资金占比']]
            txt_df[''] = ''
        model = single_account_str.split('(')[-1].split('_')[0]
        model_capital = special_treatment_capital
        txt_df['任务资金'] = round(txt_df['金额比例']/100 * model_capital)
        txt_df['昨收'] = close.iloc[-1].loc[txt_df['股票代码'].astype(int)].tolist()
        txt_df['总股数'] = ((txt_df['任务资金']/txt_df['昨收'])//100*100).tolist()
        txt_df.dropna(inplace = True)
        personal_txt_df = txt_df[['股票代码','股票名称','任务资金','总股数']]
        txt_df = txt_df[['股票代码','股票名称','总股数']]
        txt_df['总股数'] = txt_df['总股数'].astype(int)
        txt_df['产品标识'] = account_code
        txt_df['买入日期'] = special_treatment_date
        personal_txt_df['模型'] = model
        personal_txt_df['产品标识'] = account_code
        personal_txt_df['买入日期'] = special_treatment_date
        full_special_treatment = pd.concat([full_special_treatment, txt_df])
        full_special_treatment_personal = pd.concat([full_special_treatment_personal, personal_txt_df])
        print(type(model_capital), model_capital)
        print()
full_special_treatment.to_excel('2handadd.xlsx', index = False, encoding = 'gbk')
full_special_treatment_personal.to_excel(str(int_today)+'_2handadd.xlsx', index = False, encoding = 'gbk')

In [ ]:
full = full_special_treatment_personal.groupby(['产品标识','股票代码']).sum().reset_index()
full['股票名称'] = code_to_name.loc[full['股票代码'].astype(int)].tolist()

In [ ]:
for account_code in full['产品标识'].unique().tolist():
    account_name = pd.read_excel('手动补仓模型要买资金.xlsx')[pd.read_excel('手动补仓模型要买资金.xlsx')['代码'] == account_code].iloc[0,1]
    sub = full[full['产品标识'] == account_code]
    sub[''] = ''
    sub = sub[['股票代码','股票名称','总股数','任务资金','',]]
    sub[['总股数','任务资金',]] = sub[['总股数','任务资金',]].astype(int)
    sub['任务资金'] = round(sub['任务资金']/sub['任务资金'].sum()*100,4)
    sub.columns = ['证券代码','证券名称','每股份数','金额比例','']
    sub.to_excel(account_code+account_name+' '+str(int_today)+'_handadd.xlsx',index=False,encoding='gbk')

In [ ]:
full_special_treatment_buy = full_special_treatment_personal.copy()
full_special_treatment_buy['任务股数'] = np.nan
full_special_treatment_buy['方向'] = '多'
full_special_treatment_buy = full_special_treatment_buy[['产品标识','股票代码','任务资金','任务股数','方向']]

In [ ]:
full_special_treatment_buy.to_excel('总任务管理表_买入_补.xlsx', index = False, encoding = 'gbk')

In [ ]:
#（总任务管理表_卖出_补 这个就是一个空文件）

In [ ]:
pd.concat([full_special_treatment_buy, pd.read_excel('总任务管理表_卖出_补.xlsx')]).to_excel('总任务管理表_补.xlsx', index = False, encoding = 'gbk')

In [ ]:
output = full_special_treatment_personal[full_special_treatment_personal['总股数'] != 0]

In [ ]:
sell_date = []
for i in range(len(output)):
    m = output.iloc[i,4]
    h = preset_hold_days(m)
    b = output.iloc[i,6]
    sell_date.append(int(get_n_week_date_after(str(b),h-1)))
output['卖出'] = sell_date
output['1'] = 1
output['2'] = 1
output = output[['产品标识','股票代码','卖出','1','2','总股数']]
output['产品标识'] = output['产品标识'].str.replace('RB','HQ')
output['股票代码'] = output['股票代码'].astype(int)
output.to_excel(str(int_today)+'_handadd_add_to_hold.xlsx', index = False, encoding = 'gbk')

In [ ]:
#制作管理表

In [ ]:
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')

In [ ]:
full_txt_df.to_excel('未修改_买入.xlsx',encoding='gbk',index=False)

In [ ]:
#如果需要手动修改，直接按格式出去改excel文件即可

In [ ]:
full_txt_df = pd.read_excel('未修改_买入.xlsx',encoding='gbk')

In [ ]:
shiyan_guanlibiao = full_txt_df.loc[:,['产品标识','证券代码','任务资金','任务股数','方向','模型']].rename(columns={'证券代码':'股票代码'})
mairu = pd.concat([full_txt_df.loc[:,['产品标识','证券代码','任务资金','任务股数','方向']].rename(columns={'证券代码':'股票代码'}),\
           pd.read_excel('买入_未完成_'+str(int_yesterday)+'.xlsx')])
zong = pd.concat([full_txt_df.loc[:,['产品标识','证券代码','任务资金','任务股数','方向']].rename(columns={'证券代码':'股票代码'}),\
           pd.read_excel('买入_未完成_'+str(int_yesterday)+'.xlsx'),\
           pd.read_excel('总任务管理表_卖出.xlsx')])

In [ ]:
shiyan_guanlibiao.to_excel('实验_总任务管理表_买入_'+today_nodot+'.xlsx',index = False, encoding = 'gbk')

In [ ]:
mairu.to_excel('总任务管理表_买入.xlsx',index = False, encoding = 'gbk')

In [ ]:
zong.to_excel('总任务管理表.xlsx',index = False, encoding = 'gbk')

In [ ]:
#morebuy
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')
close_data = pd.read_csv('tsdata_nonright_'+str(int_yesterday)+'.csv', encoding='gbk',index_col=0).iloc[:,5].replace('<NIL>',np.nan).astype(float)
close_data.index = close_data.index.str[2:].astype(int)
buy = pd.read_excel('总任务管理表_买入_合_'+today_nodot+'.xlsx')
sxf = pd.read_excel('sxf策略'+today_nodot[-4:]+'买.xlsx')
more_buy = pd.read_excel('实验_总任务管理表_买入_未完成_'+str(int_yesterday)+'.xlsx')
more_buy['close'] = close_data.loc[more_buy['股票代码']].tolist()
more_buy['任务股数'] = more_buy['完成资金']/more_buy['close']
more_buy = more_buy[more_buy['模型'] > int_today]
def process(df, valve_capital, valve_share):
    if (df.sum()['完成资金']>= valve_capital) & (df.sum()['任务股数']>= valve_share):
        return 1
    else:
        return 0
#去掉2000块、100股以下的
capitalnshare_filter = more_buy.groupby(['产品标识','股票代码']).apply(process, 2000, 100)
more_buy = more_buy.set_index(['产品标识','股票代码'])
more_buy['完成资金'] = (more_buy['完成资金']*capitalnshare_filter).replace(0,np.nan)
more_buy.dropna(inplace = True)
more_buy.reset_index(inplace = True)
more_buy['任务资金'] = more_buy['完成资金']
more_buy['完成资金'] = np.nan
more_buy = more_buy.loc[:,['产品标识','股票代码','模型','任务资金','完成资金']]

In [ ]:
#一些核对项，正常应该输出True,0,空的Series等

In [ ]:
len(buy.groupby(['产品标识','股票代码']).sum().sort_index()['任务资金']) == \
len(pd.concat([full_txt_df.rename(columns={'证券代码':'股票代码'}), sxf, more_buy]).groupby(['产品标识','股票代码']).sum().sort_index()['任务资金'])

In [ ]:
(pd.concat([buy.groupby(['产品标识','股票代码']).sum().sort_index()['任务资金'], pd.concat([full_txt_df.rename(columns={'证券代码':'股票代码'}),\
    sxf, more_buy]).groupby(['产品标识','股票代码']).sum().sort_index()['任务资金']],axis=1).fillna(0).iloc[:,0] - \
pd.concat([buy.groupby(['产品标识','股票代码']).sum().sort_index()['任务资金'], pd.concat([full_txt_df.rename(columns={'证券代码':'股票代码'}),\
    sxf, more_buy]).groupby(['产品标识','股票代码']).sum().sort_index()['任务资金']],axis=1).fillna(0).iloc[:,1])\
.replace(0,np.nan).replace(1,np.nan).replace(-1,np.nan).replace(2,np.nan).replace(-2,np.nan)\
.dropna().sort_values().reset_index()['产品标识'].unique()

In [ ]:
(pd.concat([buy.groupby(['产品标识','股票代码']).sum().sort_index()['任务资金'], pd.concat([full_txt_df.rename(columns={'证券代码':'股票代码'}),\
    sxf, more_buy]).groupby(['产品标识','股票代码']).sum().sort_index()['任务资金']],axis=1).fillna(0).iloc[:,0] - \
pd.concat([buy.groupby(['产品标识','股票代码']).sum().sort_index()['任务资金'], pd.concat([full_txt_df.rename(columns={'证券代码':'股票代码'}),\
    sxf, more_buy]).groupby(['产品标识','股票代码']).sum().sort_index()['任务资金']],axis=1).fillna(0).iloc[:,1])\
.replace(0,np.nan).replace(1,np.nan).replace(-1,np.nan).replace(2,np.nan).replace(-2,np.nan).dropna().sort_index()

In [ ]:
(pd.concat([buy.groupby(['产品标识','股票代码']).sum().sort_index()['任务资金'], pd.concat([full_txt_df.rename(columns={'证券代码':'股票代码'}),\
    sxf, more_buy]).groupby(['产品标识','股票代码']).sum().sort_index()['任务资金']],axis=1).fillna(0).iloc[:,0] - \
pd.concat([buy.groupby(['产品标识','股票代码']).sum().sort_index()['任务资金'], pd.concat([full_txt_df.rename(columns={'证券代码':'股票代码'}),\
    sxf, more_buy]).groupby(['产品标识','股票代码']).sum().sort_index()['任务资金']],axis=1).fillna(0).iloc[:,1])\
.replace(0,np.nan).replace(1,np.nan).replace(-1,np.nan).replace(2,np.nan).replace(-2,np.nan).dropna().sort_index().reset_index()['股票代码'].unique()

In [ ]:
sxf['模型'] = 'sxf'
updated_buy = pd.concat([full_txt_df.loc[:,['产品标识','证券代码','任务资金','任务股数','方向','模型']]\
.rename(columns={'证券代码':'股票代码'}), sxf, more_buy])

In [ ]:
(pd.concat([buy.groupby(['产品标识','股票代码']).sum().sort_index()['任务资金'], \
            updated_buy.groupby(['产品标识','股票代码']).sum().sort_index()['任务资金']],axis=1).fillna(0).iloc[:,0] - \
pd.concat([buy.groupby(['产品标识','股票代码']).sum().sort_index()['任务资金'], \
           updated_buy.groupby(['产品标识','股票代码']).sum().sort_index()['任务资金']],axis=1).fillna(0).iloc[:,1])\
.replace(0,np.nan).replace(1,np.nan).replace(-1,np.nan).replace(2,np.nan).replace(-2,np.nan).dropna().sort_values()

In [ ]:
len(buy.groupby(['产品标识','股票代码']).sum()) == \
len(updated_buy.groupby(['产品标识','股票代码']).sum())

In [ ]:
updated_buy = updated_buy.reset_index()[~updated_buy.reset_index()['产品标识'].isin(['HQGS2','HQZX3','HQHT5','HQCT5','HQHT13'])]

In [ ]:
updated_buy.to_excel('实验_总任务管理表_买入_'+today_nodot+'.xlsx',index = False, encoding = 'gbk')

In [ ]:
updated_buy['模型'].unique()

In [ ]:
updated_buy = updated_buy.set_index(['产品标识','股票代码'])

In [ ]:
updated_buy = updated_buy.dropna(subset = ['任务资金'])

In [ ]:
updated_buy_unfinished = updated_buy.copy()

### 早上运行到这里结束

In [ ]:
os.chdir(r'C:/Users/bj/Desktop/刘泽丹/Code/')

In [ ]:
end_of_day_result = pd.read_excel('任务管理_管理员_'+today.replace('.','-')+'.xlsx')
#end_of_day_result = pd.read_excel('任务管理_管理员_'+today.replace('.','-')+'-集合.xlsx')

In [ ]:
(end_of_day_result.groupby(['产品标识','方向']).sum().xs('多',level=1)['完成资金']/\
end_of_day_result.groupby(['产品标识','方向']).sum().xs('多',level=1)['任务资金']).sort_values()

In [ ]:
(end_of_day_result.groupby(['产品标识','方向']).sum().xs('空',level=1)['完成股数']/\
end_of_day_result.groupby(['产品标识','方向']).sum().xs('空',level=1)['任务股数']).sort_values()

In [ ]:
end_of_day_result = end_of_day_result[end_of_day_result['方向'] == '多']
end_of_day_result = end_of_day_result.groupby(['产品标识','股票代码']).sum()

In [ ]:
def process(df):
    return df.set_index('模型')['任务资金']/df.set_index('模型')['任务资金'].sum()
updated_buy = updated_buy.reset_index().groupby(['产品标识','股票代码']).apply(process).reset_index().set_index(['产品标识','股票代码'])

In [ ]:
[i for i in updated_buy.index if i not in end_of_day_result.index]

In [ ]:
[i for i in end_of_day_result.index if i not in updated_buy.index]

In [ ]:
updated_buy['完成资金'] = (updated_buy['任务资金'] * end_of_day_result['完成资金']).dropna()
updated_buy['完成股数'] = (updated_buy['任务资金'] * end_of_day_result['完成股数']).dropna()

In [ ]:
updated_buy.sort_values('完成股数')

In [ ]:
def process(df):
    zero = [0] * len(df)
    zero[(df%100).tolist().index(max(df%100))] = round((df%100).sum())
    return pd.Series(zero,index=df.index) + df//100*100
updated_buy['完成股数'] = updated_buy['完成股数'].groupby(level=[0,1]).apply(process)

In [ ]:
np.isclose(updated_buy['完成资金'].sum(), end_of_day_result['完成资金'].sum())

In [ ]:
updated_buy['完成股数'].sum() == end_of_day_result['完成股数'].sum()

In [ ]:
(updated_buy.groupby(level=[0,1]).sum()['完成股数'] != end_of_day_result.groupby(level=[0,1]).sum()['完成股数']).sum()

In [ ]:
updated_buy.reset_index().to_excel('实验_总任务管理表_买入_完成_'+today_nodot+'.xlsx',index = False, encoding = 'gbk')

In [ ]:
end_of_day_result_unfinished = pd.read_excel('任务管理_管理员_'+today.replace('.','-')+'.xlsx')
end_of_day_result_unfinished = end_of_day_result_unfinished[end_of_day_result_unfinished['方向'] == '多']
end_of_day_result_unfinished = end_of_day_result_unfinished.groupby(['产品标识','股票代码']).sum()
end_of_day_result_unfinished = (end_of_day_result_unfinished['任务资金'] - end_of_day_result_unfinished['完成资金'])
warning_part = end_of_day_result_unfinished[end_of_day_result_unfinished<0]
end_of_day_result_unfinished.name = '任务资金'
end_of_day_result_unfinished = end_of_day_result_unfinished[end_of_day_result_unfinished>0].reset_index().groupby(['产品标识','股票代码']).sum()

In [ ]:
def process(df):
    return df.set_index('模型')['任务资金']/df.set_index('模型')['任务资金'].sum()
updated_buy_unfinished = updated_buy_unfinished.reset_index().groupby(['产品标识','股票代码'])\
.apply(process).reset_index().set_index(['产品标识','股票代码'])

In [ ]:
updated_buy_unfinished['完成资金'] = end_of_day_result_unfinished['任务资金'] * updated_buy_unfinished['任务资金']

In [ ]:
np.isclose(updated_buy_unfinished['完成资金'].sum(), end_of_day_result_unfinished['任务资金'].sum())

In [ ]:
updated_buy_unfinished['完成资金'].sum()

In [ ]:
#replace date
updated_buy_unfinished['完成资金'] = updated_buy_unfinished['完成资金']//1
for mm in preset_n_days_model_combined(range(100)):
    updated_buy_unfinished['模型'] = updated_buy_unfinished['模型'].replace(mm,int(get_n_week_date_after(today_nodot,preset_hold_days(mm)-1)))
for dd in range(100):
    updated_buy_unfinished['模型'] = updated_buy_unfinished['模型'].replace(dd,int(get_n_week_date_after(today_nodot,dd-1)))

In [ ]:
updated_buy_unfinished['完成资金'] = updated_buy_unfinished['完成资金'].replace(0,np.nan)
updated_buy_unfinished = updated_buy_unfinished.dropna(subset = ['完成资金'])

In [ ]:
updated_buy_unfinished.groupby(level=0).sum()['完成资金'].sort_values()

In [ ]:
updated_buy_unfinished = updated_buy_unfinished.reset_index()[~updated_buy_unfinished.reset_index()['产品标识']\
    .isin([])].set_index(['产品标识','股票代码'])
updated_buy_unfinished = updated_buy_unfinished.reset_index()[~updated_buy_unfinished.reset_index()['股票代码']\
    .isin([])].set_index(['产品标识','股票代码'])

In [ ]:
updated_buy_unfinished.reset_index().to_excel('实验_总任务管理表_买入_未完成_'+today_nodot+'.xlsx',index = False, encoding = 'gbk')

In [ ]:
result = updated_buy_unfinished[(updated_buy_unfinished['模型']>int(get_n_week_date_after(str(int_today))))]

In [ ]:
result = result.groupby(['产品标识','股票代码']).sum()['完成资金']
result.name = '任务资金'
result = result.reset_index()
result['任务股数'] = ''
result['方向'] = '多'
close_data = pd.read_csv('tsdata_nonright_'+today_nodot+'.csv', encoding='gbk',index_col=0).iloc[:,5].replace('<NIL>',np.nan).astype(float)
close_data.index = close_data.index.str[2:].astype(int)
#close_data = pd.read_csv('close.csv',index_col=0).iloc[-1]
#close_data.index = close_data.index.astype(int)
result['close'] = close_data.loc[result['股票代码']].tolist()
result['任务股数'] = result['任务资金']/result['close']
result = result[(result['任务股数']>=100)&(result['任务资金']>=2000)]
result['任务股数'] = ''
result = result.loc[:,['产品标识','股票代码','任务资金','任务股数','方向']]

In [ ]:
result.to_excel('买入_未完成_'+today_nodot+'.xlsx',index = False, encoding = 'gbk')

In [ ]:
result.groupby('产品标识').sum().sort_values('任务资金')

In [ ]:
result['任务资金'].sum()

In [ ]:
result.groupby('股票代码').sum()['任务资金'].sort_values()

In [ ]:
result.groupby('股票代码').sum()['任务资金'].sort_values()[result.groupby('股票代码').sum()['任务资金'].sort_values()>1000000].index.tolist()